In [11]:
# Install noveum-trace from PyPI and testing dependencies
%pip install noveum-trace python-dotenv openai anthropic
%pip install --upgrade noveum-trace pip


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [12]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file if it exists
# Note: Using robust path detection since __file__ is not available in Jupyter notebooks
try:
    # Try to get current working directory
    current_dir = os.getcwd()
    print(f"📁 Current directory: {current_dir}")
    
    # Look for .env file in current directory and parent directories
    env_file_found = False
    search_dir = current_dir
    
    for _ in range(5):  # Search up to 5 levels up
        env_file = os.path.join(search_dir, '.env')
        if os.path.exists(env_file):
            print(f"📄 Found .env file: {env_file}")
            load_dotenv(env_file)
            env_file_found = True
            break
        search_dir = os.path.dirname(search_dir)
        if search_dir == os.path.dirname(search_dir):  # Reached root
            break
    
    if not env_file_found:
        print("ℹ️  No .env file found - continuing without it")
        
except Exception as e:
    print(f"⚠️  Error loading .env file: {e}")
    print("ℹ️  Continuing without .env file")

# Set up environment variables for testing
# Replace with your actual API key or set in .env file
if not os.getenv('NOVEUM_API_KEY'):
    # For testing purposes, you can set a dummy API key
    # In production, use your actual Noveum API key
    os.environ['NOVEUM_API_KEY'] = 'test-api-key-for-demo'
    print("⚠️  Using dummy API key for testing. Set NOVEUM_API_KEY environment variable for production use.")
else:
    print("✅ NOVEUM_API_KEY found in environment")

# Optional: Set OpenAI API key for LLM examples
if not os.getenv('OPENAI_API_KEY'):
    print("ℹ️  OPENAI_API_KEY not found. LLM examples will use mock responses.")
else:
    print("✅ OPENAI_API_KEY found in environment")

print("\n📋 Environment Variables Status:")
print(f"NOVEUM_API_KEY: {'✓' if os.getenv('NOVEUM_API_KEY') else '✗'}")
print(f"OPENAI_API_KEY: {'✓' if os.getenv('OPENAI_API_KEY') else '✗'}")


✅ Loaded .env from: ../../../.env
✅ NOVEUM_API_KEY found in environment
✅ OPENAI_API_KEY found in environment

📋 Environment Variables Status:
NOVEUM_API_KEY: ✓
OPENAI_API_KEY: ✓


## 🔄 FLUSH HELPER: Automatic Trace Sending

To ensure all traces are sent immediately to your endpoint, we'll create a helper function that can be called after any traced operation.


In [13]:
# 🔄 FLUSH HELPER FUNCTIONS FOR IMMEDIATE TRACE SENDING

def flush_traces(operation_name="Operation"):
    """
    Helper function to flush traces immediately to endpoint.
    Call this after any traced operation to ensure traces are sent right away.
    """
    try:
        noveum_trace.flush()
        print(f"📤 ✅ {operation_name} traces flushed successfully")
    except Exception as e:
        print(f"📤 ⚠️  {operation_name} flush warning: {e}")

def auto_flush_decorator(func):
    """
    Decorator that automatically flushes traces after function execution.
    Use this for any function that contains traced operations.
    """
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        flush_traces(func.__name__)
        return result
    return wrapper

# Test the flush helper
print("🔧 Flush helper functions initialized")
print("📋 Usage:")
print("  - Call flush_traces('operation_name') after any traced operation")
print("  - Use @auto_flush_decorator on functions containing traced operations")
print("  - This ensures immediate trace sending to your endpoint")


🔧 Flush helper functions initialized
📋 Usage:
  - Call flush_traces('operation_name') after any traced operation
  - Use @auto_flush_decorator on functions containing traced operations
  - This ensures immediate trace sending to your endpoint


In [14]:
import noveum_trace
from noveum_trace import trace, trace_agent, trace_llm, trace_tool
import logging

# Enable detailed logging to debug transport issues
logging.basicConfig(level=logging.DEBUG)
transport_logger = logging.getLogger('noveum_trace.transport')
transport_logger.setLevel(logging.DEBUG)
ENDPOINT = "https://noveum.ai/api"

# Initialize the SDK with proper endpoint configuration
try:
    # IMPORTANT: The SDK will append "/v1/traces" to your endpoint
    # So "https://noveum-trace.free.beeceptor.com" becomes "https://noveum-trace.free.beeceptor.com/v1/traces"
    noveum_trace.init(
        api_key=os.getenv('NOVEUM_API_KEY'),
        project="jupyter-test-project", 
        environment="development",
        endpoint=ENDPOINT,  # SDK will add /v1/traces automatically
        debug=True,  # Enable debug mode for testing
        
        # Transport configuration for better reliability
        transport_config={
            "timeout": 10,           # 10 second timeout
            "retry_attempts": 2,     # Retry failed requests 2 times
            "batch_size": 10,        # Smaller batches for demo
            "batch_timeout": 2.0,    # Send batches every 2 seconds
            "compression": False     # Disable compression for debugging
        },
        
        # Tracing configuration
        tracing_config={
            "sample_rate": 1.0,      # Trace 100% of operations
            "capture_errors": True,
            "capture_stack_traces": True
        }
    )
    
    print("✅ Noveum Trace SDK initialized successfully!")
    print("📊 Project: jupyter-test-project")
    print("🔧 Environment: development") 
    print(f"🌐 Endpoint: {ENDPOINT}/v1/traces (auto-appended)")
    print("🔍 Debug logging enabled - check console for HTTP request details")
    
    # Get the current configuration to verify settings
    config = noveum_trace.get_config()
    print(f"📋 Config verified - Endpoint: {config.transport.endpoint}")
    print(f"📦 Batch size: {config.transport.batch_size}")
    print(f"⏱️  Batch timeout: {config.transport.batch_timeout}s")
    
except Exception as e:
    print(f"❌ Error initializing SDK: {e}")
    print("Continuing with demo - traces will be logged locally")
    import traceback
    traceback.print_exc()


✅ Noveum Trace SDK initialized successfully!
📊 Project: jupyter-test-project
🔧 Environment: development
🌐 Endpoint: https://noveum.ai/api/v1/traces (auto-appended)
🔍 Debug logging enabled - check console for HTTP request details
📋 Config verified - Endpoint: https://noveum.ai/api
📦 Batch size: 100
⏱️  Batch timeout: 5.0s


In [15]:
# 🔄 FLUSH AFTER LLM CALL TRACING
# This ensures the @trace_llm decorator traces are sent immediately

flush_traces("LLM Call Tracing (call_language_model)")


INFO:noveum_trace.transport.http_transport:HTTP transport flush completed
INFO:noveum_trace.core.client:Flushed all pending traces


📤 ✅ LLM Call Tracing (call_language_model) traces flushed successfully


In [16]:
# Mock LLM responses for testing (replace with actual API calls if you have keys)
def mock_openai_call(prompt: str, model: str = "gpt-4") -> str:
    """Mock OpenAI API call for testing."""
    responses = [
        "This is a mock response from the language model.",
        "Here's a simulated AI response for testing purposes.",
        "Mock LLM output to demonstrate tracing functionality."
    ]
    time.sleep(0.3)  # Simulate API call latency
    return random.choice(responses)

@trace_llm
def call_language_model(prompt: str, model: str = "gpt-4") -> str:
    """Call a language model with tracing."""
    print(f"🤖 Calling {model} with prompt: {prompt[:50]}...")

    # Use real OpenAI API if available, otherwise use mock
    if os.getenv('OPENAI_API_KEY'):
        try:
            import openai
            client = openai.OpenAI()
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=100
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"⚠️  OpenAI API call failed: {e}. Using mock response.")
            return mock_openai_call(prompt, model)
    else:
        print("📝 Using mock LLM response (no API key provided)")
        return mock_openai_call(prompt, model)

# Test LLM tracing
prompt = "Explain the benefits of observability in AI systems."
response = call_language_model(prompt)
print(f"\n🎯 LLM Response: {response}")


DEBUG:noveum_trace.core.client:Started trace: e44bb7d3-5e7f-4628-ba73-6967bcc4079b
DEBUG:noveum_trace.core.client:Started span: 64343ce6-19d6-4f2a-8014-c23cf8dfc319 in trace: e44bb7d3-5e7f-4628-ba73-6967bcc4079b
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'idempotency_key': 'stainless-python-retry-863b5391-c6e4-43fd-a05d-ea0722156917', 'json_data': {'messages': [{'role': 'user', 'content': 'Explain the benefits of observability in AI systems.'}], 'model': 'gpt-4', 'max_tokens': 100}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=5.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x115d89590>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x10732b530> server_hostn

🤖 Calling gpt-4 with prompt: Explain the benefits of observability in AI system...


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Sat, 19 Jul 2025 19:31:03 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'magicapi-inc'), (b'openai-processing-ms', b'3255'), (b'openai-project', b'proj_5py2FGvCF1HwegoK03gVwlEm'), (b'openai-version', b'2020-10-01'), (b'x-envoy-upstream-service-time', b'3259'), (b'x-ratelimit-limit-requests', b'5000'), (b'x-ratelimit-limit-tokens', b'40000'), (b'x-ratelimit-remaining-requests', b'4999'), (b'x-ratelimit-remaining-tokens', b'39985'), (b'x-ratelimit-reset-requests', b'12ms'), (b'x-ratelimit-reset-tokens', b'22ms'), (b'x-request-id', b'req_812e8f22c5cc77585c07cc436434dda9'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'cf-cache-status', b'DYNAMIC'), (b'Set-Cookie', b'__cf_bm=nQutduGj.hXxDdQQk8kWHzOlrr


🎯 LLM Response: 1. Improved Decision Making: Observability in AI helps understand the process of decision-making. It provides more insight into the mechanisms that AI uses to reach a conclusion, which can help improve the accuracy and effectiveness of the AI's function.

2. Troubleshooting and Root Cause Analysis: With observability, one can easily detect and diagnose the problems in the AI system. It helps identify the root cause of any issue in the system, making it convenient to rectify the error and prevent its reocc


In [7]:
# 🔄 FLUSH AFTER ENHANCED LLM TRACING
# This ensures all enhanced LLM traces (Anthropic, OpenAI, Google) are sent immediately

flush_traces("Enhanced LLM Tracing (Anthropic + OpenAI + Google)")


INFO:noveum_trace.transport.http_transport:HTTP transport flush completed
INFO:noveum_trace.core.client:Flushed all pending traces


📤 ✅ Enhanced LLM Tracing (Anthropic + OpenAI + Google) traces flushed successfully


In [8]:
# 🔄 FLUSH AFTER RETRIEVAL SYSTEM TRACING  
# This ensures all @trace_retrieval decorator traces are sent immediately

flush_traces("Retrieval System Tracing (Vector + Keyword + Hybrid)")


INFO:noveum_trace.transport.http_transport:HTTP transport flush completed
INFO:noveum_trace.core.client:Flushed all pending traces


📤 ✅ Retrieval System Tracing (Vector + Keyword + Hybrid) traces flushed successfully


In [9]:
# Enhanced LLM Tracing Examples

# Test different LLM providers with comprehensive metadata
@trace_llm(provider="anthropic", capture_tokens=True, estimate_costs=True)
def call_anthropic(prompt: str, model: str = "claude-3-haiku") -> str:
    """Call Anthropic Claude with tracing."""
    print(f"🧠 Calling {model} with prompt: {prompt[:50]}...")
    
    # Mock Anthropic response
    time.sleep(0.4)
    responses = [
        "Observability in AI systems provides critical insights into model behavior and performance.",
        "Tracing AI workflows enables debugging, optimization, and compliance monitoring.",
        "Comprehensive monitoring helps ensure AI system reliability and user trust."
    ]
    return random.choice(responses)

# Test with custom metadata and tags
@trace_llm(
    provider="openai", 
    capture_prompts=True, 
    capture_completions=True,
    metadata={"experiment": "demo", "version": "1.0"},
    tags={"environment": "notebook", "user": "demo"}
)
def call_llm_with_metadata(prompt: str, model: str = "gpt-4") -> str:
    """LLM call with custom metadata and tags."""
    print(f"📋 Enhanced LLM call with metadata: {prompt[:40]}...")
    time.sleep(0.3)
    return f"Enhanced response for: {prompt[:20]}..."

# Test Google AI provider
@trace_llm(provider="google", capture_tokens=True, redact_pii=True)
def call_google_ai(prompt: str, model: str = "gemini-pro") -> str:
    """Call Google AI with PII redaction."""
    print(f"🟢 Calling {model} with PII protection: {prompt[:40]}...")
    time.sleep(0.5)
    return "Google AI response with PII redaction enabled for sensitive data handling."

# Test various LLM providers
print("🤖 Testing Enhanced LLM Tracing...")

anthropic_response = call_anthropic("What are the key benefits of AI observability?")
print(f"\n🧠 Anthropic Response: {anthropic_response}")

metadata_response = call_llm_with_metadata("Summarize the importance of AI monitoring")
print(f"\n📋 Enhanced Response: {metadata_response}")

google_response = call_google_ai("How does tracing help with AI compliance?")
print(f"\n🟢 Google AI Response: {google_response}")

print("\n✅ Enhanced LLM tracing completed!")


DEBUG:noveum_trace.core.client:Started trace: b9cdd133-5987-4fb7-b7d5-ad9eedf9a403
DEBUG:noveum_trace.core.client:Started span: b503a69a-c088-4270-8866-d097bfe42cae in trace: b9cdd133-5987-4fb7-b7d5-ad9eedf9a403
DEBUG:noveum_trace.core.client:Finished span: b503a69a-c088-4270-8866-d097bfe42cae
DEBUG:noveum_trace.transport.http_transport:Trace b9cdd133-5987-4fb7-b7d5-ad9eedf9a403 queued for export
DEBUG:noveum_trace.core.client:Finished trace: b9cdd133-5987-4fb7-b7d5-ad9eedf9a403


🤖 Testing Enhanced LLM Tracing...
🧠 Calling claude-3-haiku with prompt: What are the key benefits of AI observability?...


NameError: name 'time' is not defined

In [10]:
# 🔄 FLUSH AFTER ENHANCED MULTI-AGENT SYSTEM TRACING
# This ensures all @trace_agent decorator traces are sent immediately

flush_traces("Enhanced Multi-Agent System (Data Analyst + Content Curator + Synthesis + Orchestrator)")


INFO:noveum_trace.transport.http_transport:HTTP transport flush completed
INFO:noveum_trace.core.client:Flushed all pending traces


📤 ✅ Enhanced Multi-Agent System (Data Analyst + Content Curator + Synthesis + Orchestrator) traces flushed successfully


In [54]:
# 🔄 FLUSH AFTER CONTEXT MANAGERS AND STREAMING
# This ensures all context manager traces are sent immediately

flush_traces("Context Managers and Streaming (trace_llm_call + trace_agent_operation + trace_operation + streaming)")


INFO:noveum_trace.transport.http_transport:HTTP transport flush completed
INFO:noveum_trace.core.client:Flushed all pending traces


📤 ✅ Context Managers and Streaming (trace_llm_call + trace_agent_operation + trace_operation + streaming) traces flushed successfully


In [55]:
# Import the retrieval decorator
from noveum_trace import trace_retrieval

# Vector search with comprehensive tracing
@trace_retrieval(
    retrieval_type="vector_search",
    index_name="knowledge_base",
    capture_query=True,
    capture_results=True,
    capture_scores=True,
    metadata={"index_version": "v2.1", "embedding_model": "text-embedding-ada-002"}
)
def vector_search(query: str, top_k: int = 5) -> Dict[str, Any]:
    """Perform vector search with tracing."""
    print(f"🔍 Vector Search: Finding top {top_k} results for '{query}'")
    
    # Simulate vector search
    time.sleep(0.3)
    
    # Mock search results with scores
    results = []
    for i in range(top_k):
        results.append({
            "document_id": f"doc_{i+1}",
            "content": f"Relevant content for '{query}' - document {i+1}",
            "score": 0.95 - (i * 0.1),
            "metadata": {"source": f"source_{i+1}.pdf", "page": i+1}
        })
    
    search_result = {
        "query": query,
        "total_results": top_k,
        "results": results,
        "search_time_ms": 300,
        "index_stats": {"total_docs": 10000, "dimensions": 1536}
    }
    
    print(f"✅ Found {len(results)} relevant documents")
    return search_result

# Keyword search with metadata capture
@trace_retrieval(
    retrieval_type="keyword_search",
    index_name="text_corpus",
    capture_metadata=True,
    tags={"search_type": "fulltext", "language": "en"}
)
def keyword_search(query: str, filters: Optional[Dict] = None) -> Dict[str, Any]:
    """Perform keyword search with filtering."""
    print(f"🔎 Keyword Search: '{query}' with filters: {filters}")
    
    time.sleep(0.2)
    
    # Mock keyword search results
    results = [
        {"doc_id": "kw_1", "title": "AI Observability Guide", "snippet": "...observability in AI..."},
        {"doc_id": "kw_2", "title": "Tracing Best Practices", "snippet": "...tracing methodologies..."},
        {"doc_id": "kw_3", "title": "Monitoring AI Systems", "snippet": "...monitoring strategies..."}
    ]
    
    return {
        "query": query,
        "filters": filters or {},
        "results": results,
        "total_matches": len(results)
    }

# Test retrieval operations
print("🔍 Testing Retrieval System Tracing...")

# Test vector search
vector_result = vector_search("benefits of AI observability", top_k=3)
print(f"\n🔍 Vector Search Results: {len(vector_result['results'])} documents")

# Test keyword search with filters
keyword_result = keyword_search("AI monitoring", filters={"category": "technical", "year": 2024})
print(f"\n🔎 Keyword Search Results: {keyword_result['total_matches']} matches")

print("\n✅ Retrieval tracing completed!")


DEBUG:noveum_trace.core.client:Started trace: 8d691518-5983-4eda-85c0-64964c7faadc
DEBUG:noveum_trace.core.client:Started span: ee709a5e-f8c9-41ea-a74d-9b34e73964f8 in trace: 8d691518-5983-4eda-85c0-64964c7faadc


🔍 Testing Retrieval System Tracing...
🔍 Vector Search: Finding top 3 results for 'benefits of AI observability'


DEBUG:noveum_trace.core.client:Finished span: ee709a5e-f8c9-41ea-a74d-9b34e73964f8
DEBUG:noveum_trace.transport.http_transport:Trace 8d691518-5983-4eda-85c0-64964c7faadc queued for export
DEBUG:noveum_trace.core.client:Finished trace: 8d691518-5983-4eda-85c0-64964c7faadc
DEBUG:noveum_trace.core.client:Started trace: b6e3f513-edec-4c18-bc53-8ad58295eccb
DEBUG:noveum_trace.core.client:Started span: f40f6a43-282f-49e9-a4b3-0fb1482a6cf2 in trace: b6e3f513-edec-4c18-bc53-8ad58295eccb


✅ Found 3 relevant documents

🔍 Vector Search Results: 3 documents
🔎 Keyword Search: 'AI monitoring' with filters: {'category': 'technical', 'year': 2024}


DEBUG:noveum_trace.core.client:Finished span: f40f6a43-282f-49e9-a4b3-0fb1482a6cf2
DEBUG:noveum_trace.transport.http_transport:Trace b6e3f513-edec-4c18-bc53-8ad58295eccb queued for export
DEBUG:noveum_trace.core.client:Finished trace: b6e3f513-edec-4c18-bc53-8ad58295eccb



🔎 Keyword Search Results: 3 matches

✅ Retrieval tracing completed!


In [56]:
# Enhanced Multi-Agent System Examples
# Import required types to ensure they're available in this cell
from typing import Dict, Any, List

# Specialized agents with different roles and capabilities
@trace_agent(
    agent_id="data_analyst",
    role="analyst",
    agent_type="specialist",
    capabilities=["data_analysis", "statistical_modeling", "visualization"],
    capture_reasoning=True,
    metadata={"specialization": "quantitative_analysis", "confidence_threshold": 0.8}
)
def data_analyst_agent(data: Dict[str, Any]) -> Dict[str, Any]:
    """Specialized data analysis agent."""
    print(f"📊 Data Analyst: Analyzing dataset with {len(data.get('samples', []))} samples")
    
    time.sleep(0.5)
    
    # Mock data analysis
    analysis = {
        "agent_id": "data_analyst",
        "analysis_type": "quantitative",
        "findings": {
            "data_quality": 0.92,
            "pattern_confidence": 0.87,
            "anomalies_detected": 3,
            "recommendations": [
                "Data quality is high with 92% confidence",
                "3 anomalies detected requiring investigation",
                "Statistical patterns show strong correlation"
            ]
        },
        "reasoning_steps": [
            "Loaded and validated input data",
            "Applied statistical analysis methods", 
            "Identified patterns and anomalies",
            "Generated confidence-based recommendations"
        ],
        "processing_time": 0.5
    }
    
    print(f"✅ Analysis complete: {analysis['findings']['data_quality']:.2f} quality score")
    return analysis

@trace_agent(
    agent_id="content_curator",
    role="curator",
    agent_type="content_specialist", 
    capabilities=["content_filtering", "quality_assessment", "summarization"],
    capture_tools=True
)
def content_curator_agent(content_list: List[Dict]) -> Dict[str, Any]:
    """Content curation and quality assessment agent."""
    print(f"📝 Content Curator: Processing {len(content_list)} content items")
    
    time.sleep(0.4)
    
    # Mock content curation using tools
    high_quality_content = []
    for i, content in enumerate(content_list):
        if i < 3:  # Mock: keep first 3 as high quality
            high_quality_content.append({
                **content,
                "quality_score": 0.9 - (i * 0.05),
                "curation_reason": "Meets quality standards"
            })
    
    curation_result = {
        "agent_id": "content_curator",
        "input_count": len(content_list),
        "curated_count": len(high_quality_content),
        "curated_content": high_quality_content,
        "tools_used": ["quality_scorer", "content_filter", "summarizer"],
        "curation_metrics": {
            "retention_rate": len(high_quality_content) / len(content_list),
            "average_quality": sum(item["quality_score"] for item in high_quality_content) / len(high_quality_content)
        }
    }
    
    print(f"✅ Curated {len(high_quality_content)}/{len(content_list)} items")
    return curation_result

# Test the enhanced multi-agent system
print("🤖 Testing Enhanced Multi-Agent System...")

# Mock data for demonstration
sample_data = {"samples": [f"sample_{i}" for i in range(100)]}
sample_content = [
    {"id": 1, "title": "AI Observability", "content": "Content about observability"},
    {"id": 2, "title": "Tracing Systems", "content": "Content about tracing"},
    {"id": 3, "title": "Monitoring Tools", "content": "Content about monitoring"},
    {"id": 4, "title": "Low Quality", "content": "Poor content"}
]

# Test individual agents
analyst_result = data_analyst_agent(sample_data)
print(f"\n📊 Data Analysis: {analyst_result['findings']['data_quality']:.2f} quality score")

curator_result = content_curator_agent(sample_content)
print(f"\n📝 Content Curation: {curator_result['curated_count']}/{curator_result['input_count']} items retained")

print("\n✅ Enhanced multi-agent system testing completed!")


DEBUG:noveum_trace.core.client:Started trace: b08455d3-5ddb-4cdd-b139-0b6eebf38cda
DEBUG:noveum_trace.core.client:Started span: 0ea8b562-a000-4657-af18-0a22cc1090ac in trace: b08455d3-5ddb-4cdd-b139-0b6eebf38cda


🤖 Testing Enhanced Multi-Agent System...
📊 Data Analyst: Analyzing dataset with 100 samples


DEBUG:noveum_trace.core.client:Finished span: 0ea8b562-a000-4657-af18-0a22cc1090ac
DEBUG:noveum_trace.transport.http_transport:Trace b08455d3-5ddb-4cdd-b139-0b6eebf38cda queued for export
DEBUG:noveum_trace.core.client:Finished trace: b08455d3-5ddb-4cdd-b139-0b6eebf38cda
DEBUG:noveum_trace.core.client:Started trace: 18454ea0-845b-4b1e-84d6-e759b19d7c13
DEBUG:noveum_trace.core.client:Started span: df81263e-bc0a-4f26-94d9-1528842f5e8a in trace: 18454ea0-845b-4b1e-84d6-e759b19d7c13


✅ Analysis complete: 0.92 quality score

📊 Data Analysis: 0.92 quality score
📝 Content Curator: Processing 4 content items


DEBUG:noveum_trace.core.client:Finished span: df81263e-bc0a-4f26-94d9-1528842f5e8a
DEBUG:noveum_trace.transport.http_transport:Trace 18454ea0-845b-4b1e-84d6-e759b19d7c13 queued for export
DEBUG:noveum_trace.core.client:Finished trace: 18454ea0-845b-4b1e-84d6-e759b19d7c13


✅ Curated 3/4 items

📝 Content Curation: 3/4 items retained

✅ Enhanced multi-agent system testing completed!


In [57]:
# Import context managers and streaming features
from noveum_trace import (
    trace_llm_call, trace_agent_operation, trace_operation, 
    streaming_llm, trace_streaming, ThreadContext
)
from typing import Dict, Any, Iterator

# Context Manager Examples - Inline Tracing

def process_user_query_with_context_managers(user_input: str) -> str:
    """Demonstrate inline tracing with context managers."""
    print(f"🔄 Processing user query: '{user_input[:40]}...'")
    
    # Some preprocessing (not traced)
    cleaned_input = user_input.strip().lower()
    
    # Trace just the LLM call using context manager
    with trace_llm_call(model="gpt-4", provider="openai", operation="query_processing") as span:
        print("🤖 Making LLM call within context manager...")
        time.sleep(0.4)
        
        # Mock LLM response
        response = f"Processed response for: {cleaned_input[:30]}..."
        
        # Add custom attributes to the span
        span.set_attributes({
            "llm.input_length": len(cleaned_input),
            "llm.output_length": len(response),
            "llm.processing_type": "query_understanding"
        })
        
        print(f"✅ LLM response generated: {len(response)} characters")
    
    # Post-processing (not traced)
    final_response = f"Final: {response}"
    print(f"📤 Final response: {final_response[:50]}...")
    
    return final_response

# Agent operation context manager
def agent_task_with_context_manager(task: str) -> Dict[str, Any]:
    """Demonstrate agent operation tracing with context manager."""
    print(f"🤖 Agent Task: '{task}'")
    
    with trace_agent_operation(
        agent_type="task_agent", 
        operation="task_execution",
        capabilities=["task_planning", "execution", "monitoring"]
    ) as span:
        print("⚙️  Executing agent task...")
        time.sleep(0.3)
        
        # Mock agent work
        result = {
            "task": task,
            "status": "completed",
            "steps_executed": 5,
            "success_rate": 0.95
        }
        
        # Add agent-specific attributes
        span.set_attributes({
            "agent.task_complexity": "medium",
            "agent.steps_executed": result["steps_executed"],
            "agent.success_rate": result["success_rate"]
        })
        
        print(f"✅ Agent task completed with {result['success_rate']:.1%} success rate")
    
    return result

# Generic operation context manager (FIXED SYNTAX)
def complex_operation_with_tracing() -> Dict[str, Any]:
    """Demonstrate generic operation tracing."""
    print("🔧 Starting complex operation...")
    
    # CORRECT SYNTAX: trace_operation(operation_name, attributes=...)
    with trace_operation("complex_data_processing", 
                        attributes={"operation_type": "data_pipeline", "complexity": "high"}) as span:
        # Step 1: Data loading
        print("📥 Step 1: Loading data...")
        time.sleep(0.2)
        span.set_attributes({"step": "data_loading", "records_loaded": 1000})
        
        # Step 2: Processing
        print("⚙️  Step 2: Processing data...")
        time.sleep(0.3)
        span.set_attributes({"step": "processing", "records_processed": 950})
        
        # Step 3: Output
        print("📤 Step 3: Generating output...")
        time.sleep(0.1)
        span.set_attributes({"step": "output", "records_output": 950})
        
        result = {
            "operation": "complex_data_processing",
            "input_records": 1000,
            "processed_records": 950,
            "success": True
        }
        
        print("✅ Complex operation completed successfully")
    
    return result

# Test context managers
print("🔄 Testing Context Managers...")

print("\\n1️⃣ Context Manager Examples:")
query_result = process_user_query_with_context_managers("What are the benefits of AI observability?")

agent_result = agent_task_with_context_manager("Analyze system performance metrics")

operation_result = complex_operation_with_tracing()

print("\\n✅ Context managers testing completed!")


DEBUG:noveum_trace.core.client:Started trace: e2873aac-2abd-4aa5-9702-1948f1a93847
DEBUG:noveum_trace.core.client:Started span: a517c511-81b0-4105-9455-3ec4da2a3fdf in trace: e2873aac-2abd-4aa5-9702-1948f1a93847


🔄 Testing Context Managers...
\n1️⃣ Context Manager Examples:
🔄 Processing user query: 'What are the benefits of AI observabilit...'
🤖 Making LLM call within context manager...


DEBUG:noveum_trace.core.client:Finished span: a517c511-81b0-4105-9455-3ec4da2a3fdf
DEBUG:noveum_trace.transport.http_transport:Trace e2873aac-2abd-4aa5-9702-1948f1a93847 queued for export
DEBUG:noveum_trace.core.client:Finished trace: e2873aac-2abd-4aa5-9702-1948f1a93847
DEBUG:noveum_trace.core.client:Started trace: f499b45b-9753-48a3-849d-7b356391eed4
DEBUG:noveum_trace.core.client:Started span: c56bdd26-1c21-447b-83e6-45622ec13751 in trace: f499b45b-9753-48a3-849d-7b356391eed4


✅ LLM response generated: 57 characters
📤 Final response: Final: Processed response for: what are the benefi...
🤖 Agent Task: 'Analyze system performance metrics'
⚙️  Executing agent task...


DEBUG:noveum_trace.core.client:Finished span: c56bdd26-1c21-447b-83e6-45622ec13751
DEBUG:noveum_trace.transport.http_transport:Trace f499b45b-9753-48a3-849d-7b356391eed4 queued for export
DEBUG:noveum_trace.core.client:Finished trace: f499b45b-9753-48a3-849d-7b356391eed4
DEBUG:noveum_trace.core.client:Started trace: 51348c5a-ff42-443b-81de-90945a4d5797
DEBUG:noveum_trace.core.client:Started span: 3a720185-4daf-4717-8e7f-902ab6e7d8fc in trace: 51348c5a-ff42-443b-81de-90945a4d5797


✅ Agent task completed with 95.0% success rate
🔧 Starting complex operation...
📥 Step 1: Loading data...
⚙️  Step 2: Processing data...


DEBUG:noveum_trace.core.client:Finished span: 3a720185-4daf-4717-8e7f-902ab6e7d8fc
DEBUG:noveum_trace.transport.http_transport:Trace 51348c5a-ff42-443b-81de-90945a4d5797 queued for export
DEBUG:noveum_trace.core.client:Finished trace: 51348c5a-ff42-443b-81de-90945a4d5797


📤 Step 3: Generating output...
✅ Complex operation completed successfully
\n✅ Context managers testing completed!


In [58]:
# Import advanced features
from noveum_trace import (
    auto_instrument, get_instrumented_libraries, is_instrumented,
    create_traced_openai_client, create_traced_agent, TracedOpenAIClient,
    start_trace, start_span, get_current_trace, get_current_span
)

# Auto-Instrumentation Examples

def test_auto_instrumentation():
    """Test automatic instrumentation of libraries."""
    print("🔧 Testing Auto-Instrumentation...")
    
    # Check available instrumentations
    try:
        available = noveum_trace.get_available_instrumentations()
        print(f"📦 Available instrumentations: {available}")
    except Exception as e:
        print(f"ℹ️  Available instrumentations: {e}")
    
    # Enable auto-instrumentation for OpenAI (if not already enabled)
    try:
        if not is_instrumented("openai"):
            print("🔌 Enabling OpenAI auto-instrumentation...")
            auto_instrument("openai")
            print("✅ OpenAI auto-instrumentation enabled")
        else:
            print("✅ OpenAI already instrumented")
    except Exception as e:
        print(f"ℹ️  Auto-instrumentation: {e}")
    
    # Check instrumented libraries
    try:
        instrumented = get_instrumented_libraries()
        print(f"🔍 Currently instrumented: {instrumented}")
    except Exception as e:
        print(f"ℹ️  Instrumented libraries: {e}")
    
    return []

# Proxy Objects for Enhanced Control

def test_traced_openai_client():
    """Test traced OpenAI client proxy."""
    print("🔄 Testing Traced OpenAI Client...")
    
    # Create traced OpenAI client (even without real API key)
    try:
        traced_client = create_traced_openai_client(
            api_key="mock-key-for-demo",
            trace_completions=True,
            trace_embeddings=True,
            capture_content=True
        )
        print("✅ Traced OpenAI client created")
        
        # Mock a call (won't actually work without real API key)
        print("🤖 Simulating traced OpenAI call...")
        # In real usage: response = traced_client.chat.completions.create(...)
        print("ℹ️  Would automatically trace all OpenAI API calls")
        
    except Exception as e:
        print(f"ℹ️  Traced client demo: {e}")

def test_traced_agent_proxy():
    """Test traced agent proxy for enhanced agent monitoring."""
    print("🤖 Testing Traced Agent Proxy...")
    
    # Mock agent class
    class MockAgent:
        def __init__(self, name: str):
            self.name = name
        
        def think(self, problem: str) -> str:
            time.sleep(0.2)
            return f"Thinking about: {problem}"
        
        def act(self, action: str) -> str:
            time.sleep(0.3)
            return f"Performing action: {action}"
        
        def plan(self, goal: str) -> List[str]:
            time.sleep(0.4)
            return [f"Step 1 for {goal}", f"Step 2 for {goal}", f"Step 3 for {goal}"]
    
    # Create traced agent proxy with CORRECT PARAMETERS
    original_agent = MockAgent("demo_agent")
    traced_agent = create_traced_agent(
        agent=original_agent,
        agent_type="traced_demo_agent",  # CORRECT: agent_type instead of agent_id
        capabilities=["thinking", "acting", "planning"],
        trace_config={"capture_inputs": True, "capture_outputs": True}
    )
    
    print("✅ Traced agent proxy created")
    
    # Test traced methods
    print("🧠 Testing traced agent methods...")
    
    thought = traced_agent.think("How to improve AI observability")
    print(f"💭 Think result: {thought}")
    
    action = traced_agent.act("Implement monitoring dashboard")
    print(f"⚡ Action result: {action}")
    
    plan = traced_agent.plan("Enhance system reliability")
    print(f"📋 Plan result: {len(plan)} steps")

# Manual Span Creation and Management

def test_manual_tracing():
    """Test manual trace and span creation."""
    print("🔍 Testing Manual Tracing...")
    
    try:
        # Start a manual trace
        trace = start_trace("manual_demo_trace")
        print(f"✅ Started trace: {trace.trace_id}")
        
        # Create nested spans manually
        with trace.span("parent_operation") as parent_span:
            parent_span.set_attributes({
                "operation.type": "parent",
                "operation.importance": "high"
            })
            print("📊 Parent span created")
            
            # Child span 1
            with parent_span.create_child_span("child_operation_1") as child1:
                child1.set_attributes({
                    "operation.type": "child",
                    "child.number": 1
                })
                time.sleep(0.2)
                print("🔹 Child span 1 completed")
            
            # Child span 2  
            with parent_span.create_child_span("child_operation_2") as child2:
                child2.set_attributes({
                    "operation.type": "child",
                    "child.number": 2,
                    "child.data_processed": 500
                })
                time.sleep(0.3)
                print("🔹 Child span 2 completed")
            
            print("📊 Parent operation completed")
        
        # Finish trace
        trace.finish()
        print(f"✅ Manual trace completed: {trace.trace_id}")
        
    except Exception as e:
        print(f"ℹ️  Manual tracing demo: {e}")

# Error Handling and Edge Cases

def test_error_handling():
    """Test error handling and edge cases."""
    print("⚠️  Testing Error Handling...")
    
    # Test error capture in traced function
    @noveum_trace.trace(capture_errors=True, capture_stack_trace=True)
    def operation_with_error(should_fail: bool = False):
        if should_fail:
            raise ValueError("This is a demo error for testing")
        return "Success!"
    
    # Test successful operation
    try:
        result = operation_with_error(should_fail=False)
        print(f"✅ Successful operation: {result}")
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
    
    # Test error capture
    try:
        result = operation_with_error(should_fail=True)
        print(f"Unexpected success: {result}")
    except ValueError as e:
        print(f"✅ Error captured successfully: {e}")

# Run all advanced feature tests
print("🚀 Testing Auto-Instrumentation and Advanced Features...")

print("\\n1️⃣ Auto-Instrumentation:")
instrumented_libs = test_auto_instrumentation()

print("\\n2️⃣ Proxy Objects:")
test_traced_openai_client()
test_traced_agent_proxy()

print("\\n3️⃣ Manual Tracing:")
test_manual_tracing()

print("\\n4️⃣ Error Handling:")
test_error_handling()

print("\\n✅ Advanced features testing completed!")


DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
/var/folders/7d/c1z608491jq84m9f40phz5rh0000gn/T/ipykernel_46328/3792679582.py:25: UserWarning: Failed to instrument openai: Failed to instrument openai: Ambiguous use of module client; please set `openai.api_type` or the `OPENAI_API_TYPE` environment variable to `openai` or `azure`
  auto_instrument("openai")
DEBUG:noveum_trace.core.client:Started trace: a63d4111-539e-4103-b6c5-f69cd87f9e35
DEBUG:noveum_trace.core.client:Started span: bc4e62cc-c2f1-489e-8bb9-c78571f5ebd5 in trace: a63d4111-539e-4103-b6c5-f69cd87f9e35


🚀 Testing Auto-Instrumentation and Advanced Features...
\n1️⃣ Auto-Instrumentation:
🔧 Testing Auto-Instrumentation...
📦 Available instrumentations: ['openai', 'anthropic', 'langchain']
🔌 Enabling OpenAI auto-instrumentation...
✅ OpenAI auto-instrumentation enabled
🔍 Currently instrumented: []
\n2️⃣ Proxy Objects:
🔄 Testing Traced OpenAI Client...
ℹ️  Traced client demo: create_traced_openai_client() got an unexpected keyword argument 'api_key'
🤖 Testing Traced Agent Proxy...
✅ Traced agent proxy created
🧠 Testing traced agent methods...


DEBUG:noveum_trace.core.client:Finished span: bc4e62cc-c2f1-489e-8bb9-c78571f5ebd5
DEBUG:noveum_trace.transport.http_transport:Trace a63d4111-539e-4103-b6c5-f69cd87f9e35 queued for export
DEBUG:noveum_trace.core.client:Finished trace: a63d4111-539e-4103-b6c5-f69cd87f9e35
DEBUG:noveum_trace.core.client:Started trace: 0d430ae2-ba17-41d6-85f4-3927eec068fa
DEBUG:noveum_trace.core.client:Started span: 13f39075-bb24-4823-a4bf-50ddeb073967 in trace: 0d430ae2-ba17-41d6-85f4-3927eec068fa


💭 Think result: Thinking about: How to improve AI observability


DEBUG:noveum_trace.core.client:Finished span: 13f39075-bb24-4823-a4bf-50ddeb073967
DEBUG:noveum_trace.transport.http_transport:Trace 0d430ae2-ba17-41d6-85f4-3927eec068fa queued for export
DEBUG:noveum_trace.core.client:Finished trace: 0d430ae2-ba17-41d6-85f4-3927eec068fa
DEBUG:noveum_trace.core.client:Started trace: dba07d70-2b05-4f4a-9456-42268d9430a9
DEBUG:noveum_trace.core.client:Started span: 9fe0c703-fb4f-4a19-84a3-587135ef0756 in trace: dba07d70-2b05-4f4a-9456-42268d9430a9


⚡ Action result: Performing action: Implement monitoring dashboard


DEBUG:noveum_trace.core.client:Finished span: 9fe0c703-fb4f-4a19-84a3-587135ef0756
DEBUG:noveum_trace.transport.http_transport:Trace dba07d70-2b05-4f4a-9456-42268d9430a9 queued for export
DEBUG:noveum_trace.core.client:Finished trace: dba07d70-2b05-4f4a-9456-42268d9430a9
DEBUG:noveum_trace.core.client:Started trace: 2e0dfbda-b28b-4f3f-93bd-8a455b51d964
DEBUG:noveum_trace.core.client:Started span: 4a530205-d24c-46ee-9652-64e06c5167d9 in trace: 2e0dfbda-b28b-4f3f-93bd-8a455b51d964
DEBUG:noveum_trace.core.client:Finished span: 4a530205-d24c-46ee-9652-64e06c5167d9
DEBUG:noveum_trace.core.client:Started span: 5ea813b8-b014-4d03-adb2-eb6356173b7b in trace: 2e0dfbda-b28b-4f3f-93bd-8a455b51d964
DEBUG:noveum_trace.core.client:Finished span: 5ea813b8-b014-4d03-adb2-eb6356173b7b


📋 Plan result: 3 steps
\n3️⃣ Manual Tracing:
🔍 Testing Manual Tracing...
✅ Started trace: 2e0dfbda-b28b-4f3f-93bd-8a455b51d964
ℹ️  Manual tracing demo: 'Trace' object has no attribute 'span'
\n4️⃣ Error Handling:
⚠️  Testing Error Handling...
✅ Successful operation: Success!
✅ Error captured successfully: This is a demo error for testing
\n✅ Advanced features testing completed!


In [59]:
# Enhanced LLM Tracing Examples

# Test different LLM providers with comprehensive metadata
@trace_llm(provider="anthropic", capture_tokens=True, estimate_costs=True)
def call_anthropic(prompt: str, model: str = "claude-3-haiku") -> str:
    """Call Anthropic Claude with tracing."""
    print(f"🧠 Calling {model} with prompt: {prompt[:50]}...")
    
    # Mock Anthropic response
    time.sleep(0.4)
    responses = [
        "Observability in AI systems provides critical insights into model behavior and performance.",
        "Tracing AI workflows enables debugging, optimization, and compliance monitoring.",
        "Comprehensive monitoring helps ensure AI system reliability and user trust."
    ]
    return random.choice(responses)

# Test with custom metadata and tags
@trace_llm(
    provider="openai", 
    capture_prompts=True, 
    capture_completions=True,
    metadata={"experiment": "demo", "version": "1.0"},
    tags={"environment": "notebook", "user": "demo"}
)
def call_llm_with_metadata(prompt: str, model: str = "gpt-4") -> str:
    """LLM call with custom metadata and tags."""
    print(f"📋 Enhanced LLM call with metadata: {prompt[:40]}...")
    time.sleep(0.3)
    return f"Enhanced response for: {prompt[:20]}..."

# Test Google AI provider
@trace_llm(provider="google", capture_tokens=True, redact_pii=True)
def call_google_ai(prompt: str, model: str = "gemini-pro") -> str:
    """Call Google AI with PII redaction."""
    print(f"🟢 Calling {model} with PII protection: {prompt[:40]}...")
    time.sleep(0.5)
    return "Google AI response with PII redaction enabled for sensitive data handling."

# Test various LLM providers
print("🤖 Testing Enhanced LLM Tracing...")

anthropic_response = call_anthropic("What are the key benefits of AI observability?")
print(f"\n🧠 Anthropic Response: {anthropic_response}")

metadata_response = call_llm_with_metadata("Summarize the importance of AI monitoring")
print(f"\n📋 Enhanced Response: {metadata_response}")

google_response = call_google_ai("How does tracing help with AI compliance?")
print(f"\n🟢 Google AI Response: {google_response}")

print("\n✅ Enhanced LLM tracing completed!")


DEBUG:noveum_trace.core.client:Started span: cf6c8189-5654-484a-9954-720453617a36 in trace: 2e0dfbda-b28b-4f3f-93bd-8a455b51d964


🤖 Testing Enhanced LLM Tracing...
🧠 Calling claude-3-haiku with prompt: What are the key benefits of AI observability?...


DEBUG:noveum_trace.core.client:Finished span: cf6c8189-5654-484a-9954-720453617a36
DEBUG:noveum_trace.core.client:Started span: a3da9f89-e154-4ba0-8d95-f12ccd27faa0 in trace: 2e0dfbda-b28b-4f3f-93bd-8a455b51d964



🧠 Anthropic Response: Comprehensive monitoring helps ensure AI system reliability and user trust.
📋 Enhanced LLM call with metadata: Summarize the importance of AI monitorin...


DEBUG:noveum_trace.core.client:Finished span: a3da9f89-e154-4ba0-8d95-f12ccd27faa0
DEBUG:noveum_trace.core.client:Started span: 1f400432-b762-4f94-bf9c-ee05098fbf89 in trace: 2e0dfbda-b28b-4f3f-93bd-8a455b51d964



📋 Enhanced Response: Enhanced response for: Summarize the import...
🟢 Calling gemini-pro with PII protection: How does tracing help with AI compliance...


DEBUG:noveum_trace.core.client:Finished span: 1f400432-b762-4f94-bf9c-ee05098fbf89



🟢 Google AI Response: Google AI response with PII redaction enabled for sensitive data handling.

✅ Enhanced LLM tracing completed!


In [60]:
# 🔄 FLUSH AFTER CORRECTED TRACE_OPERATION EXAMPLES
# This ensures all corrected context manager traces are sent immediately

flush_traces("Corrected trace_operation Examples (Fixed Syntax)")


DEBUG:noveum_trace.transport.http_transport:Trace 2e0dfbda-b28b-4f3f-93bd-8a455b51d964 queued for export
DEBUG:noveum_trace.core.client:Finished trace: 2e0dfbda-b28b-4f3f-93bd-8a455b51d964
DEBUG:urllib3.connectionpool:http://localhost:3000 "POST /api/v1/traces HTTP/1.1" 200 None
DEBUG:noveum_trace.transport.http_transport:Successfully sent batch of 9 traces
DEBUG:noveum_trace.transport.batch_processor:Sent batch of 9 traces
INFO:noveum_trace.transport.http_transport:HTTP transport flush completed
INFO:noveum_trace.core.client:Flushed all pending traces


📤 ✅ Corrected trace_operation Examples (Fixed Syntax) traces flushed successfully


In [61]:
# Import the retrieval decorator and missing typing imports
from noveum_trace import trace_retrieval
from typing import Dict, Any, Optional, List, Iterator

# Vector search with comprehensive tracing
@trace_retrieval(
    retrieval_type="vector_search",
    index_name="knowledge_base",
    capture_query=True,
    capture_results=True,
    capture_scores=True,
    metadata={"index_version": "v2.1", "embedding_model": "text-embedding-ada-002"}
)
def vector_search(query: str, top_k: int = 5) -> Dict[str, Any]:
    """Perform vector search with tracing."""
    print(f"🔍 Vector Search: Finding top {top_k} results for '{query}'")
    
    # Simulate vector search
    time.sleep(0.3)
    
    # Mock search results with scores
    results = []
    for i in range(top_k):
        results.append({
            "document_id": f"doc_{i+1}",
            "content": f"Relevant content for '{query}' - document {i+1}",
            "score": 0.95 - (i * 0.1),
            "metadata": {"source": f"source_{i+1}.pdf", "page": i+1}
        })
    
    search_result = {
        "query": query,
        "total_results": top_k,
        "results": results,
        "search_time_ms": 300,
        "index_stats": {"total_docs": 10000, "dimensions": 1536}
    }
    
    print(f"✅ Found {len(results)} relevant documents")
    return search_result

# Keyword search with metadata capture
@trace_retrieval(
    retrieval_type="keyword_search",
    index_name="text_corpus",
    capture_metadata=True,
    tags={"search_type": "fulltext", "language": "en"}
)
def keyword_search(query: str, filters: Optional[Dict] = None) -> Dict[str, Any]:
    """Perform keyword search with filtering."""
    print(f"🔎 Keyword Search: '{query}' with filters: {filters}")
    
    time.sleep(0.2)
    
    # Mock keyword search results
    results = [
        {"doc_id": "kw_1", "title": "AI Observability Guide", "snippet": "...observability in AI..."},
        {"doc_id": "kw_2", "title": "Tracing Best Practices", "snippet": "...tracing methodologies..."},
        {"doc_id": "kw_3", "title": "Monitoring AI Systems", "snippet": "...monitoring strategies..."}
    ]
    
    return {
        "query": query,
        "filters": filters or {},
        "results": results,
        "total_matches": len(results)
    }

# Hybrid search combining vector and keyword
@trace_retrieval(
    retrieval_type="hybrid_search",
    index_name="hybrid_index",
    capture_query=True,
    capture_results=True,
    capture_scores=True
)
def hybrid_search(query: str, alpha: float = 0.5) -> Dict[str, Any]:
    """Perform hybrid search combining vector and keyword search."""
    print(f"🔗 Hybrid Search: '{query}' with alpha={alpha}")
    
    time.sleep(0.4)
    
    # Simulate hybrid search by combining both approaches
    vector_results = vector_search(query, top_k=3)
    keyword_results = keyword_search(query)
    
    # Mock hybrid ranking
    hybrid_results = []
    for i, result in enumerate(vector_results["results"][:2]):
        hybrid_results.append({
            "document_id": result["document_id"],
            "content": result["content"],
            "vector_score": result["score"],
            "keyword_score": 0.8 - (i * 0.1),
            "combined_score": (result["score"] * alpha) + ((0.8 - i * 0.1) * (1 - alpha)),
            "source": "hybrid"
        })
    
    return {
        "query": query,
        "alpha": alpha,
        "results": hybrid_results,
        "total_results": len(hybrid_results),
        "search_strategy": "vector + keyword fusion"
    }

# Test all retrieval operations
print("🔍 Testing Retrieval System Tracing...")

# Test vector search
vector_result = vector_search("benefits of AI observability", top_k=3)
print(f"\n🔍 Vector Search Results: {len(vector_result['results'])} documents")

# Test keyword search with filters
keyword_result = keyword_search("AI monitoring", filters={"category": "technical", "year": 2024})
print(f"\n🔎 Keyword Search Results: {keyword_result['total_matches']} matches")

# Test hybrid search
hybrid_result = hybrid_search("observability tracing systems", alpha=0.7)
print(f"\n🔗 Hybrid Search Results: {len(hybrid_result['results'])} combined results")

print("\n✅ Retrieval tracing completed!")


DEBUG:noveum_trace.core.client:Started trace: 3e4e26f0-ba34-4e6d-b349-c3621d25b2ad
DEBUG:noveum_trace.core.client:Started span: 90d82ebd-0bd2-42bb-9a52-d2f2dd21c1c6 in trace: 3e4e26f0-ba34-4e6d-b349-c3621d25b2ad


🔍 Testing Retrieval System Tracing...
🔍 Vector Search: Finding top 3 results for 'benefits of AI observability'


DEBUG:noveum_trace.core.client:Finished span: 90d82ebd-0bd2-42bb-9a52-d2f2dd21c1c6
DEBUG:noveum_trace.transport.http_transport:Trace 3e4e26f0-ba34-4e6d-b349-c3621d25b2ad queued for export
DEBUG:noveum_trace.core.client:Finished trace: 3e4e26f0-ba34-4e6d-b349-c3621d25b2ad
DEBUG:noveum_trace.core.client:Started trace: 3fb52d31-ba85-4ac6-a327-7c2a263a24aa
DEBUG:noveum_trace.core.client:Started span: 69695bd5-f43a-4f89-881c-f6e013197f97 in trace: 3fb52d31-ba85-4ac6-a327-7c2a263a24aa


✅ Found 3 relevant documents

🔍 Vector Search Results: 3 documents
🔎 Keyword Search: 'AI monitoring' with filters: {'category': 'technical', 'year': 2024}


DEBUG:noveum_trace.core.client:Finished span: 69695bd5-f43a-4f89-881c-f6e013197f97
DEBUG:noveum_trace.transport.http_transport:Trace 3fb52d31-ba85-4ac6-a327-7c2a263a24aa queued for export
DEBUG:noveum_trace.core.client:Finished trace: 3fb52d31-ba85-4ac6-a327-7c2a263a24aa
DEBUG:noveum_trace.core.client:Started trace: abaa1e3b-0561-41a1-930b-913d73260ba9
DEBUG:noveum_trace.core.client:Started span: 927840ca-b6af-4af7-9911-9e4ed84a2620 in trace: abaa1e3b-0561-41a1-930b-913d73260ba9


...

🔍 Vector Search: Finding top 3 results for 'observability tracing systems'


DEBUG:noveum_trace.core.client:Finished span: d928bc66-4c51-48bd-8879-4758ab1f2105
DEBUG:noveum_trace.core.client:Started span: f97b2f07-df1b-42ab-9478-cc493e5d4a8e in trace: abaa1e3b-0561-41a1-930b-913d73260ba9


✅ Found 3 relevant documents
🔎 Keyword Search: 'observability tracing systems' with filters: None


DEBUG:noveum_trace.core.client:Finished span: f97b2f07-df1b-42ab-9478-cc493e5d4a8e
DEBUG:noveum_trace.core.client:Finished span: 927840ca-b6af-4af7-9911-9e4ed84a2620
DEBUG:noveum_trace.transport.http_transport:Trace abaa1e3b-0561-41a1-930b-913d73260ba9 queued for export
DEBUG:noveum_trace.core.client:Finished trace: abaa1e3b-0561-41a1-930b-913d73260ba9



🔗 Hybrid Search Results: 2 combined results

✅ Retrieval tracing completed!


In [62]:
# 🔄 FLUSH AFTER CORRECTED PROXY OBJECT EXAMPLES
# This ensures all corrected proxy object traces are sent immediately

flush_traces("Corrected Proxy Objects (create_traced_agent + create_traced_openai_client)")


DEBUG:urllib3.connectionpool:http://localhost:3000 "POST /api/v1/traces HTTP/1.1" 200 None
DEBUG:noveum_trace.transport.http_transport:Successfully sent batch of 1 traces
DEBUG:noveum_trace.transport.batch_processor:Sent batch of 1 traces
INFO:noveum_trace.transport.http_transport:HTTP transport flush completed
INFO:noveum_trace.core.client:Flushed all pending traces


📤 ✅ Corrected Proxy Objects (create_traced_agent + create_traced_openai_client) traces flushed successfully


In [63]:
# Enhanced Multi-Agent System Examples
# Import missing typing if not already available
from typing import Dict, Any, List

# Specialized agents with different roles and capabilities
@trace_agent(
    agent_id="data_analyst",
    role="analyst",
    agent_type="specialist",
    capabilities=["data_analysis", "statistical_modeling", "visualization"],
    capture_reasoning=True,
    metadata={"specialization": "quantitative_analysis", "confidence_threshold": 0.8}
)
def data_analyst_agent(data: Dict[str, Any]) -> Dict[str, Any]:
    """Specialized data analysis agent."""
    print(f"📊 Data Analyst: Analyzing dataset with {len(data.get('samples', []))} samples")
    
    time.sleep(0.5)
    
    # Mock data analysis
    analysis = {
        "agent_id": "data_analyst",
        "analysis_type": "quantitative",
        "findings": {
            "data_quality": 0.92,
            "pattern_confidence": 0.87,
            "anomalies_detected": 3,
            "recommendations": [
                "Data quality is high with 92% confidence",
                "3 anomalies detected requiring investigation",
                "Statistical patterns show strong correlation"
            ]
        },
        "reasoning_steps": [
            "Loaded and validated input data",
            "Applied statistical analysis methods",
            "Identified patterns and anomalies",
            "Generated confidence-based recommendations"
        ],
        "processing_time": 0.5
    }
    
    print(f"✅ Analysis complete: {analysis['findings']['data_quality']:.2f} quality score")
    return analysis

@trace_agent(
    agent_id="content_curator",
    role="curator",
    agent_type="content_specialist",
    capabilities=["content_filtering", "quality_assessment", "summarization"],
    capture_tools=True
)
def content_curator_agent(content_list: List[Dict]) -> Dict[str, Any]:
    """Content curation and quality assessment agent."""
    print(f"📝 Content Curator: Processing {len(content_list)} content items")
    
    time.sleep(0.4)
    
    # Mock content curation using tools
    high_quality_content = []
    for i, content in enumerate(content_list):
        if i < 3:  # Mock: keep first 3 as high quality
            high_quality_content.append({
                **content,
                "quality_score": 0.9 - (i * 0.05),
                "curation_reason": "Meets quality standards"
            })
    
    curation_result = {
        "agent_id": "content_curator",
        "input_count": len(content_list),
        "curated_count": len(high_quality_content),
        "curated_content": high_quality_content,
        "tools_used": ["quality_scorer", "content_filter", "summarizer"],
        "curation_metrics": {
            "retention_rate": len(high_quality_content) / len(content_list),
            "average_quality": sum(item["quality_score"] for item in high_quality_content) / len(high_quality_content)
        }
    }
    
    print(f"✅ Curated {len(high_quality_content)}/{len(content_list)} items")
    return curation_result

@trace_agent(
    agent_id="synthesis_agent",
    role="synthesizer",
    agent_type="integration_specialist", 
    capabilities=["multi_source_synthesis", "insight_generation", "report_creation"],
    capture_inputs=True,
    capture_outputs=True
)
def synthesis_agent(analyst_data: Dict, curator_data: Dict, context: str) -> Dict[str, Any]:
    """Agent that synthesizes insights from multiple sources."""
    print(f"🔗 Synthesis Agent: Combining insights for context '{context}'")
    
    time.sleep(0.6)
    
    # Synthesize insights from multiple agents
    synthesis = {
        "agent_id": "synthesis_agent",
        "context": context,
        "input_sources": ["data_analyst", "content_curator"],
        "synthesis_insights": [
            f"Data quality score of {analyst_data['findings']['data_quality']:.2f} indicates reliable foundation",
            f"Content curation retained {curator_data['curated_count']}/{curator_data['input_count']} high-quality items",
            "Cross-analysis reveals consistent quality patterns across data and content",
            "Synthesis confidence: High based on convergent evidence"
        ],
        "combined_metrics": {
            "data_quality": analyst_data['findings']['data_quality'],
            "content_quality": curator_data['curation_metrics']['average_quality'],
            "overall_confidence": (analyst_data['findings']['pattern_confidence'] + 
                                 curator_data['curation_metrics']['average_quality']) / 2
        },
        "final_recommendation": "Proceed with high confidence based on quality convergence"
    }
    
    print(f"✅ Synthesis complete with {synthesis['combined_metrics']['overall_confidence']:.2f} confidence")
    return synthesis

# Advanced orchestrator with dependency management
@trace_agent(
    agent_id="advanced_orchestrator",
    role="coordinator", 
    agent_type="orchestrator",
    capabilities=["workflow_management", "dependency_resolution", "result_aggregation"],
    capture_reasoning=True,
    metadata={"orchestration_strategy": "parallel_with_dependencies"}
)
def advanced_orchestrator(task: str, data_context: Dict) -> Dict[str, Any]:
    """Advanced orchestrator managing complex multi-agent workflows."""
    print(f"🎭 Advanced Orchestrator: Managing workflow for '{task}'")
    
    # Phase 1: Parallel execution of independent agents
    print("\n🔄 Phase 1: Parallel Agent Execution")
    
    # Mock data for demonstration
    sample_data = {"samples": [f"sample_{i}" for i in range(100)]}
    sample_content = [
        {"id": 1, "title": "AI Observability", "content": "Content about observability"},
        {"id": 2, "title": "Tracing Systems", "content": "Content about tracing"},
        {"id": 3, "title": "Monitoring Tools", "content": "Content about monitoring"},
        {"id": 4, "title": "Low Quality", "content": "Poor content"}
    ]
    
    # Execute agents in parallel (simulated)
    analyst_result = data_analyst_agent(sample_data)
    curator_result = content_curator_agent(sample_content)
    
    # Phase 2: Synthesis based on results
    print("\\n🔗 Phase 2: Synthesis and Integration")
    synthesis_result = synthesis_agent(analyst_result, curator_result, task)
    
    # Final orchestration result
    orchestration_result = {
        "task": task,
        "orchestration_id": "adv_orch_001",
        "phases": {
            "analysis": analyst_result,
            "curation": curator_result, 
            "synthesis": synthesis_result
        },
        "workflow_metrics": {
            "total_agents": 3,
            "execution_phases": 2,
            "final_confidence": synthesis_result["combined_metrics"]["overall_confidence"],
            "workflow_success": True
        },
        "reasoning": [
            "Initiated parallel execution of specialist agents",
            "Data analyst provided quantitative insights",
            "Content curator filtered and assessed quality",
            "Synthesis agent combined multi-source insights",
            "Workflow completed with high confidence"
        ]
    }
    
    print(f"\\n✅ Advanced orchestration complete!")
    return orchestration_result

# Test the enhanced multi-agent system
print("🤖 Testing Enhanced Multi-Agent System...")

# Run the advanced workflow
task = "Comprehensive analysis of AI system observability data and content"
context_data = {"domain": "ai_observability", "priority": "high"}

workflow_result = advanced_orchestrator(task, context_data)

print("\\n🎭 Enhanced Multi-Agent Results:")
print(f"Task: {workflow_result['task']}")
print(f"Agents: {workflow_result['workflow_metrics']['total_agents']}")
print(f"Phases: {workflow_result['workflow_metrics']['execution_phases']}")
print(f"Final Confidence: {workflow_result['workflow_metrics']['final_confidence']:.2f}")
print(f"Success: {workflow_result['workflow_metrics']['workflow_success']}")

print("\\n✅ Enhanced multi-agent system testing completed!")


DEBUG:noveum_trace.core.client:Started trace: 0fd3906b-3a45-4bd5-a4c9-a1df8c50f722
DEBUG:noveum_trace.core.client:Started span: 963b00c4-9039-4c19-a586-56baa527afca in trace: 0fd3906b-3a45-4bd5-a4c9-a1df8c50f722
DEBUG:noveum_trace.core.client:Started span: 437e5143-895c-428a-b45c-aaebd113f05f in trace: 0fd3906b-3a45-4bd5-a4c9-a1df8c50f722


🤖 Testing Enhanced Multi-Agent System...
🎭 Advanced Orchestrator: Managing workflow for 'Comprehensive analysis of AI system observability data and content'

🔄 Phase 1: Parallel Agent Execution
📊 Data Analyst: Analyzing dataset with 100 samples


DEBUG:noveum_trace.core.client:Finished span: 437e5143-895c-428a-b45c-aaebd113f05f
DEBUG:noveum_trace.core.client:Started span: 708c0ec0-ec62-4b85-8483-08b3dc7209ef in trace: 0fd3906b-3a45-4bd5-a4c9-a1df8c50f722


✅ Analysis complete: 0.92 quality score
📝 Content Curator: Processing 4 content items


DEBUG:noveum_trace.core.client:Finished span: 708c0ec0-ec62-4b85-8483-08b3dc7209ef
DEBUG:noveum_trace.core.client:Started span: 5cbfe248-458b-45dc-a2e5-f3ae59e75b84 in trace: 0fd3906b-3a45-4bd5-a4c9-a1df8c50f722


✅ Curated 3/4 items
\n🔗 Phase 2: Synthesis and Integration
🔗 Synthesis Agent: Combining insights for context 'Comprehensive analysis of AI system observability data and content'


DEBUG:noveum_trace.core.client:Finished span: 5cbfe248-458b-45dc-a2e5-f3ae59e75b84
DEBUG:noveum_trace.core.client:Finished span: 963b00c4-9039-4c19-a586-56baa527afca
DEBUG:noveum_trace.transport.http_transport:Trace 0fd3906b-3a45-4bd5-a4c9-a1df8c50f722 queued for export
DEBUG:noveum_trace.core.client:Finished trace: 0fd3906b-3a45-4bd5-a4c9-a1df8c50f722


✅ Synthesis complete with 0.86 confidence
\n✅ Advanced orchestration complete!
\n🎭 Enhanced Multi-Agent Results:
Task: Comprehensive analysis of AI system observability data and content
Agents: 3
Phases: 2
Final Confidence: 0.86
Success: True
\n✅ Enhanced multi-agent system testing completed!


In [64]:
# 🔄 FLUSH AFTER ENHANCED SDK INITIALIZATION AND ENDPOINT TESTING
# This ensures the endpoint connectivity test traces are sent immediately

flush_traces("Enhanced SDK Initialization and Endpoint Testing")


DEBUG:urllib3.connectionpool:http://localhost:3000 "POST /api/v1/traces HTTP/1.1" 200 None
DEBUG:noveum_trace.transport.http_transport:Successfully sent batch of 1 traces
DEBUG:noveum_trace.transport.batch_processor:Sent batch of 1 traces
INFO:noveum_trace.transport.http_transport:HTTP transport flush completed
INFO:noveum_trace.core.client:Flushed all pending traces


📤 ✅ Enhanced SDK Initialization and Endpoint Testing traces flushed successfully


In [65]:
# Import context managers and streaming features
from noveum_trace import (
    trace_llm_call, trace_agent_operation, trace_operation, 
    streaming_llm, trace_streaming, ThreadContext
)

# Context Manager Examples - Inline Tracing

def process_user_query_with_context_managers(user_input: str) -> str:
    """Demonstrate inline tracing with context managers."""
    print(f"🔄 Processing user query: '{user_input[:40]}...'")
    
    # Some preprocessing (not traced)
    cleaned_input = user_input.strip().lower()
    
    # Trace just the LLM call using context manager
    with trace_llm_call(model="gpt-4", provider="openai", operation="query_processing") as span:
        print("🤖 Making LLM call within context manager...")
        time.sleep(0.4)
        
        # Mock LLM response
        response = f"Processed response for: {cleaned_input[:30]}..."
        
        # Add custom attributes to the span
        span.set_attributes({
            "llm.input_length": len(cleaned_input),
            "llm.output_length": len(response),
            "llm.processing_type": "query_understanding"
        })
        
        print(f"✅ LLM response generated: {len(response)} characters")
    
    # Post-processing (not traced)
    final_response = f"Final: {response}"
    print(f"📤 Final response: {final_response[:50]}...")
    
    return final_response

# Agent operation context manager
def agent_task_with_context_manager(task: str) -> Dict[str, Any]:
    """Demonstrate agent operation tracing with context manager."""
    print(f"🤖 Agent Task: '{task}'")
    
    with trace_agent_operation(
        agent_type="task_agent", 
        operation="task_execution",
        capabilities=["task_planning", "execution", "monitoring"]
    ) as span:
        print("⚙️  Executing agent task...")
        time.sleep(0.3)
        
        # Mock agent work
        result = {
            "task": task,
            "status": "completed",
            "steps_executed": 5,
            "success_rate": 0.95
        }
        
        # Add agent-specific attributes
        span.set_attributes({
            "agent.task_complexity": "medium",
            "agent.steps_executed": result["steps_executed"],
            "agent.success_rate": result["success_rate"]
        })
        
        print(f"✅ Agent task completed with {result['success_rate']:.1%} success rate")
    
    return result

# Generic operation context manager
def complex_operation_with_tracing() -> Dict[str, Any]:
    """Demonstrate generic operation tracing."""
    print("🔧 Starting complex operation...")
    
    with trace_operation(name="complex_data_processing", operation_type="data_pipeline") as span:
        # Step 1: Data loading
        print("📥 Step 1: Loading data...")
        time.sleep(0.2)
        span.set_attributes({"step": "data_loading", "records_loaded": 1000})
        
        # Step 2: Processing
        print("⚙️  Step 2: Processing data...")
        time.sleep(0.3)
        span.set_attributes({"step": "processing", "records_processed": 950})
        
        # Step 3: Output
        print("📤 Step 3: Generating output...")
        time.sleep(0.1)
        span.set_attributes({"step": "output", "records_output": 950})
        
        result = {
            "operation": "complex_data_processing",
            "input_records": 1000,
            "processed_records": 950,
            "success": True
        }
        
        print("✅ Complex operation completed successfully")
    
    return result

# Streaming LLM Examples

class MockStreamChunk:
    """Mock streaming response chunk."""
    def __init__(self, content: str):
        self.choices = [MockChoice(content)]

class MockChoice:
    """Mock choice in streaming response."""
    def __init__(self, content: str):
        self.delta = MockDelta(content)

class MockDelta:
    """Mock delta content."""
    def __init__(self, content: str):
        self.content = content

def mock_streaming_response(prompt: str) -> Iterator[MockStreamChunk]:
    """Generate mock streaming response."""
    words = f"This is a streaming response to: {prompt}. Each word comes separately.".split()
    for word in words:
        time.sleep(0.05)  # Simulate streaming delay
        yield MockStreamChunk(word + " ")

def test_streaming_with_context_manager(prompt: str) -> str:
    """Test streaming LLM with context manager."""
    print(f"🌊 Streaming LLM call: '{prompt[:30]}...'")
    
    # Create mock stream
    stream = mock_streaming_response(prompt)
    
    # Use streaming context manager
    with streaming_llm(model="gpt-4", provider="openai", operation="streaming_chat") as stream_manager:
        print("📺 Streaming response: ", end="")
        full_response = ""
        
        for chunk in stream:
            token = chunk.choices[0].delta.content
            if token:
                # Add token to stream manager for tracing
                stream_manager.add_token(token)
                print(token, end="")
                full_response += token
        
        # Add final metadata
        stream_manager.add_metadata({
            "streaming.final_length": len(full_response),
            "streaming.total_chunks": len(full_response.split())
        })
        
        print(f"\\n✅ Streaming completed: {len(full_response)} characters")
    
    return full_response.strip()

# Thread Context for Conversation Tracking

def test_thread_context_conversation() -> None:
    """Test conversation thread tracking."""
    print("💬 Testing Thread Context for Conversations...")
    
    with ThreadContext(name="demo_conversation", metadata={"session": "demo"}) as thread:
        # Simulate conversation turns
        
        # Turn 1
        thread.add_message("user", "Hello, can you help me with AI observability?")
        print("👤 User: Hello, can you help me with AI observability?")
        
        # Simulate LLM response within thread
        with trace_llm_call(model="gpt-4") as llm_span:
            time.sleep(0.3)
            response1 = "I'd be happy to help you with AI observability!"
            thread.add_message("assistant", response1)
            print(f"🤖 Assistant: {response1}")
        
        # Turn 2  
        thread.add_message("user", "What are the key components?")
        print("👤 User: What are the key components?")
        
        with trace_llm_call(model="gpt-4") as llm_span:
            time.sleep(0.4)
            response2 = "Key components include tracing, metrics, and logging."
            thread.add_message("assistant", response2)
            print(f"🤖 Assistant: {response2}")
        
        # Get thread statistics
        stats = thread.get_statistics()
        print(f"\\n📊 Thread Stats: {stats['message_count']} messages, {stats['turn_count']} turns")

# Test all context manager and streaming features
print("🔄 Testing Context Managers and Streaming...")

# Test context managers
print("\\n1️⃣ Context Manager Examples:")
query_result = process_user_query_with_context_managers("What are the benefits of AI observability?")

agent_result = agent_task_with_context_manager("Analyze system performance metrics")

operation_result = complex_operation_with_tracing()

# Test streaming
print("\\n2️⃣ Streaming Examples:")
stream_result = test_streaming_with_context_manager("Explain machine learning concepts")

# Test thread context
print("\\n3️⃣ Thread Context Examples:")
test_thread_context_conversation()

print("\\n✅ Context managers and streaming testing completed!")


DEBUG:noveum_trace.core.client:Started trace: 4b504e8b-e27b-4418-9244-b687733cbebc
DEBUG:noveum_trace.core.client:Started span: 23cf6769-ebfe-4491-960d-f76d9e174df8 in trace: 4b504e8b-e27b-4418-9244-b687733cbebc


🔄 Testing Context Managers and Streaming...
\n1️⃣ Context Manager Examples:
🔄 Processing user query: 'What are the benefits of AI observabilit...'
🤖 Making LLM call within context manager...


DEBUG:noveum_trace.core.client:Finished span: 23cf6769-ebfe-4491-960d-f76d9e174df8
DEBUG:noveum_trace.transport.http_transport:Trace 4b504e8b-e27b-4418-9244-b687733cbebc queued for export
DEBUG:noveum_trace.core.client:Finished trace: 4b504e8b-e27b-4418-9244-b687733cbebc
DEBUG:noveum_trace.core.client:Started trace: 8296746f-e52f-4b1e-949d-8a29855c53f4
DEBUG:noveum_trace.core.client:Started span: 5a33e68a-cb5a-4ef7-b969-7b556799d192 in trace: 8296746f-e52f-4b1e-949d-8a29855c53f4


✅ LLM response generated: 57 characters
📤 Final response: Final: Processed response for: what are the benefi...
🤖 Agent Task: 'Analyze system performance metrics'
⚙️  Executing agent task...


DEBUG:noveum_trace.core.client:Finished span: 5a33e68a-cb5a-4ef7-b969-7b556799d192
DEBUG:noveum_trace.transport.http_transport:Trace 8296746f-e52f-4b1e-949d-8a29855c53f4 queued for export
DEBUG:noveum_trace.core.client:Finished trace: 8296746f-e52f-4b1e-949d-8a29855c53f4


✅ Agent task completed with 95.0% success rate
🔧 Starting complex operation...


TypeError: trace_operation() missing 1 required positional argument: 'operation_name'

In [66]:
# CORRECTED trace_operation Examples
# The correct syntax is: trace_operation(operation_name, attributes=dict, tags=dict)

def test_correct_trace_operation_usage():
    """Demonstrate the CORRECT syntax for trace_operation context manager."""
    print("🔧 Testing CORRECT trace_operation usage...")
    
    # ✅ CORRECT: First parameter is operation_name (string), second is attributes (dict)
    with trace_operation("data_processing_pipeline", 
                        attributes={"operation_type": "data_pipeline", "complexity": "high"}) as span:
        # Step 1: Data loading
        print("📥 Step 1: Loading data...")
        time.sleep(0.2)
        span.set_attributes({"step": "data_loading", "records_loaded": 1000})
        
        # Step 2: Processing
        print("⚙️  Step 2: Processing data...")
        time.sleep(0.3)
        span.set_attributes({"step": "processing", "records_processed": 950})
        
        # Step 3: Output
        print("📤 Step 3: Generating output...")
        time.sleep(0.1)
        span.set_attributes({"step": "output", "records_output": 950})
        
        result = {
            "operation": "data_processing_pipeline",
            "input_records": 1000,
            "processed_records": 950,
            "success": True
        }
        
        print("✅ Complex operation completed successfully")
    
    return result

def test_correct_batch_operations():
    """Demonstrate correct trace_operation usage in batch processing."""
    print("📦 Testing CORRECT batch trace_operation usage...")
    
    operations = []
    
    for i in range(3):  # Reduced to 3 for demo
        # ✅ CORRECT: operation_name first, then attributes dict
        with trace_operation(f"batch_operation_{i}", 
                           attributes={"operation_type": "batch_demo", "batch_index": i}) as span:
            span.set_attributes({
                "batch.operation_number": i,
                "batch.total_operations": 3,
                "operation.size": "small"
            })
            time.sleep(0.1)  # Quick operations
            operations.append(f"operation_{i}")
            print(f"🔸 Batch operation {i+1}/3 completed")
    
    print(f"✅ Batch processing completed: {len(operations)} operations")
    return operations

# Test the corrected functions
print("🔧 Testing CORRECTED Context Manager Usage...")

print("\\n1️⃣ Correct trace_operation Usage:")
operation_result = test_correct_trace_operation_usage()

print("\\n2️⃣ Correct Batch Operations:")
batch_result = test_correct_batch_operations()

print(f"\\n✅ All corrected context manager examples completed!")
print(f"Operation result: {operation_result['success']}")
print(f"Batch operations: {len(batch_result)} completed")


DEBUG:noveum_trace.core.client:Started trace: ab5c73b5-6281-4057-9072-a652ab851852
DEBUG:noveum_trace.core.client:Started span: bc1d3195-e720-499e-a9e5-4ca0f6e44cd2 in trace: ab5c73b5-6281-4057-9072-a652ab851852


🔧 Testing CORRECTED Context Manager Usage...
\n1️⃣ Correct trace_operation Usage:
🔧 Testing CORRECT trace_operation usage...
📥 Step 1: Loading data...
⚙️  Step 2: Processing data...


DEBUG:noveum_trace.core.client:Finished span: bc1d3195-e720-499e-a9e5-4ca0f6e44cd2
DEBUG:noveum_trace.transport.http_transport:Trace ab5c73b5-6281-4057-9072-a652ab851852 queued for export
DEBUG:noveum_trace.core.client:Finished trace: ab5c73b5-6281-4057-9072-a652ab851852
DEBUG:noveum_trace.core.client:Started trace: c220f277-b383-4889-a59b-5936d551dff4
DEBUG:noveum_trace.core.client:Started span: d9fa3f3b-6e87-4973-acfa-eda05ead43b9 in trace: c220f277-b383-4889-a59b-5936d551dff4


📤 Step 3: Generating output...
✅ Complex operation completed successfully
\n2️⃣ Correct Batch Operations:
📦 Testing CORRECT batch trace_operation usage...


DEBUG:noveum_trace.core.client:Finished span: d9fa3f3b-6e87-4973-acfa-eda05ead43b9
DEBUG:noveum_trace.transport.http_transport:Trace c220f277-b383-4889-a59b-5936d551dff4 queued for export
DEBUG:noveum_trace.core.client:Finished trace: c220f277-b383-4889-a59b-5936d551dff4
DEBUG:noveum_trace.core.client:Started trace: 66dac688-4039-4a23-94ff-f9a31bf4b033
DEBUG:noveum_trace.core.client:Started span: de8a7c20-a98f-40a1-9d39-8c9863065544 in trace: 66dac688-4039-4a23-94ff-f9a31bf4b033
DEBUG:noveum_trace.core.client:Finished span: de8a7c20-a98f-40a1-9d39-8c9863065544
DEBUG:noveum_trace.transport.http_transport:Trace 66dac688-4039-4a23-94ff-f9a31bf4b033 queued for export
DEBUG:noveum_trace.core.client:Finished trace: 66dac688-4039-4a23-94ff-f9a31bf4b033
DEBUG:noveum_trace.core.client:Started trace: f638fb44-816e-41ec-961e-0a132d7e2c66
DEBUG:noveum_trace.core.client:Started span: 79f65cc8-6997-4f4e-8a3b-ea110fc7e52e in trace: f638fb44-816e-41ec-961e-0a132d7e2c66


🔸 Batch operation 1/3 completed
🔸 Batch operation 2/3 completed


DEBUG:noveum_trace.core.client:Finished span: 79f65cc8-6997-4f4e-8a3b-ea110fc7e52e
DEBUG:noveum_trace.transport.http_transport:Trace f638fb44-816e-41ec-961e-0a132d7e2c66 queued for export
DEBUG:noveum_trace.core.client:Finished trace: f638fb44-816e-41ec-961e-0a132d7e2c66


🔸 Batch operation 3/3 completed
✅ Batch processing completed: 3 operations
\n✅ All corrected context manager examples completed!
Operation result: True
Batch operations: 3 completed


In [67]:
# CORRECTED create_traced_agent Example
# The correct signature is: create_traced_agent(agent, agent_type, capabilities, trace_config)

def test_traced_agent_proxy_corrected():
    """Test traced agent proxy with CORRECT parameters."""
    print("🤖 Testing Traced Agent Proxy (CORRECTED)...")
    
    # Mock agent class
    class MockAgent:
        def __init__(self, name: str):
            self.name = name
        
        def think(self, problem: str) -> str:
            time.sleep(0.2)
            return f"Thinking about: {problem}"
        
        def act(self, action: str) -> str:
            time.sleep(0.3)
            return f"Performing action: {action}"
        
        def plan(self, goal: str) -> List[str]:
            time.sleep(0.4)
            return [f"Step 1 for {goal}", f"Step 2 for {goal}", f"Step 3 for {goal}"]
    
    # ✅ CORRECT: Use proper parameter names and structure
    original_agent = MockAgent("demo_agent")
    traced_agent = create_traced_agent(
        agent=original_agent,
        agent_type="traced_demo_agent",  # ✅ CORRECT: agent_type (not agent_id)
        capabilities=["thinking", "acting", "planning"],  # ✅ CORRECT: capabilities (not auto_trace_methods)
        trace_config={"capture_inputs": True, "capture_outputs": True}  # ✅ CORRECT: trace_config dict
    )
    
    print("✅ Traced agent proxy created successfully!")
    
    # Test traced methods
    print("🧠 Testing traced agent methods...")
    
    thought = traced_agent.think("How to improve AI observability")
    print(f"💭 Think result: {thought}")
    
    action = traced_agent.act("Implement monitoring dashboard")
    print(f"⚡ Action result: {action}")
    
    plan = traced_agent.plan("Enhance system reliability")
    print(f"📋 Plan result: {len(plan)} steps")
    
    return traced_agent

def test_traced_openai_client_corrected():
    """Test traced OpenAI client with CORRECT parameters."""
    print("🔄 Testing Traced OpenAI Client (CORRECTED)...")
    
    # ✅ CORRECT: Use actual OpenAI client instance, not direct parameters
    try:
        # First create a real OpenAI client (even with mock key)
        import openai
        original_client = openai.OpenAI(api_key="mock-key-for-demo")
        
        # ✅ CORRECT: Pass the client instance to the tracer
        traced_client = create_traced_openai_client(
            original_client=original_client,
            trace_config={"trace_completions": True, "capture_content": True}
        )
        print("✅ Traced OpenAI client created successfully!")
        
        # Mock a call (won't actually work without real API key)
        print("🤖 Simulating traced OpenAI call...")
        print("ℹ️  Would automatically trace all OpenAI API calls")
        
    except Exception as e:
        print(f"ℹ️  Traced client demo: {e}")
        print("📝 Note: This requires a real OpenAI client instance")

# Test the corrected functions
print("🚀 Testing CORRECTED Proxy Object Functions...")

print("\\n1️⃣ Corrected Traced Agent:")
traced_agent = test_traced_agent_proxy_corrected()

print("\\n2️⃣ Corrected Traced OpenAI Client:")
test_traced_openai_client_corrected()

print("\\n✅ All corrected proxy object examples completed!")


DEBUG:noveum_trace.core.client:Started trace: 27959aef-5e73-418d-bee3-54449738ce63
DEBUG:noveum_trace.core.client:Started span: 5b72295d-a29b-45f6-9d2b-b995cbce676b in trace: 27959aef-5e73-418d-bee3-54449738ce63


🚀 Testing CORRECTED Proxy Object Functions...
\n1️⃣ Corrected Traced Agent:
🤖 Testing Traced Agent Proxy (CORRECTED)...
✅ Traced agent proxy created successfully!
🧠 Testing traced agent methods...


DEBUG:noveum_trace.core.client:Finished span: 5b72295d-a29b-45f6-9d2b-b995cbce676b
DEBUG:noveum_trace.transport.http_transport:Trace 27959aef-5e73-418d-bee3-54449738ce63 queued for export
DEBUG:noveum_trace.core.client:Finished trace: 27959aef-5e73-418d-bee3-54449738ce63
DEBUG:noveum_trace.core.client:Started trace: 051214a6-a0e2-4ec9-92ad-171f2cb3f74b
DEBUG:noveum_trace.core.client:Started span: 4e1f7e98-8fa5-45c1-99e1-8310a16a6212 in trace: 051214a6-a0e2-4ec9-92ad-171f2cb3f74b


💭 Think result: Thinking about: How to improve AI observability


DEBUG:noveum_trace.core.client:Finished span: 4e1f7e98-8fa5-45c1-99e1-8310a16a6212
DEBUG:noveum_trace.transport.http_transport:Trace 051214a6-a0e2-4ec9-92ad-171f2cb3f74b queued for export
DEBUG:noveum_trace.core.client:Finished trace: 051214a6-a0e2-4ec9-92ad-171f2cb3f74b
DEBUG:noveum_trace.core.client:Started trace: ec189268-71b3-4710-baa2-55b546a3e443
DEBUG:noveum_trace.core.client:Started span: ee0b17b9-ef76-4ea6-9ac0-de05e3a156fc in trace: ec189268-71b3-4710-baa2-55b546a3e443


⚡ Action result: Performing action: Implement monitoring dashboard


DEBUG:noveum_trace.core.client:Finished span: ee0b17b9-ef76-4ea6-9ac0-de05e3a156fc
DEBUG:noveum_trace.transport.http_transport:Trace ec189268-71b3-4710-baa2-55b546a3e443 queued for export
DEBUG:noveum_trace.core.client:Finished trace: ec189268-71b3-4710-baa2-55b546a3e443


📋 Plan result: 3 steps
\n2️⃣ Corrected Traced OpenAI Client:
🔄 Testing Traced OpenAI Client (CORRECTED)...
✅ Traced OpenAI client created successfully!
🤖 Simulating traced OpenAI call...
ℹ️  Would automatically trace all OpenAI API calls
\n✅ All corrected proxy object examples completed!


In [68]:
# 🔧 ENHANCED SDK INITIALIZATION WITH ENDPOINT DEBUGGING
import noveum_trace
from noveum_trace import trace, trace_agent, trace_llm, trace_tool
import logging
import sys
import requests

# 🔍 COMPREHENSIVE DEBUGGING SETUP
print("🔧 Setting up enhanced debugging for transport layer...")

# Set up comprehensive logging with detailed output
logging.basicConfig(
    level=logging.DEBUG, 
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)

# Enable specific loggers for transport debugging
loggers_to_enable = [
    'noveum_trace.transport', 
    'noveum_trace.transport.http_transport',
    'urllib3.connectionpool'
]

for logger_name in loggers_to_enable:
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.DEBUG)

print("✅ Enhanced debugging enabled")

# 🌐 ENDPOINT VERIFICATION 
def verify_endpoint():
    \"\"\"Verify that the Beeceptor endpoint is reachable.\"\"\"
    endpoint = "https://noveum-trace.free.beeceptor.com"
    
    print(f"🔍 Verifying endpoint reachability: {endpoint}")
    
    try:
        response = requests.get(endpoint, timeout=10)
        print(f"✅ Base endpoint reachable - Status: {response.status_code}")
        
        # Test the actual trace endpoints
        trace_endpoints = [
            f"{endpoint}/v1/trace",   # Single trace endpoint
            f"{endpoint}/v1/traces"   # Batch trace endpoint  
        ]
        
        for test_endpoint in trace_endpoints:
            try:
                test_response = requests.head(test_endpoint, timeout=5)
                print(f"📡 {test_endpoint} - Status: {test_response.status_code}")
            except Exception as e:
                print(f"⚠️  {test_endpoint} - Error: {e}")
                
    except Exception as e:
        print(f"❌ Endpoint verification failed: {e}")
        return False
    
    return True

# Verify endpoint first
endpoint_ok = verify_endpoint()


SyntaxError: unexpected character after line continuation character (1154693373.py, line 33)

In [69]:
# 🚀 ENHANCED SDK INITIALIZATION
try:
    print("\\n🚀 Initializing Noveum Trace SDK with enhanced configuration...")
    
    # 📋 Display endpoint mapping
    base_endpoint = "https://noveum-trace.free.beeceptor.com"
    print(f"🌐 Base Endpoint: {base_endpoint}")
    print(f"📡 Single Trace Endpoint: {base_endpoint}/v1/trace")
    print(f"📦 Batch Trace Endpoint: {base_endpoint}/v1/traces")
    print(f"🔧 Transport Mode: Individual traces (batch_size=1) for better debugging")
    
    noveum_trace.init(
        api_key=os.getenv('NOVEUM_API_KEY'),
        project="jupyter-test-project",
        environment="development", 
        endpoint=base_endpoint,  # SDK will append /v1/trace or /v1/traces automatically
        debug=True,  # Enable debug mode
        
        # 🔧 Enhanced transport configuration for debugging
        transport_config={
            "timeout": 30,           # 30 second timeout (generous for debugging)
            "retry_attempts": 0,     # No retries for faster debugging feedback  
            "batch_size": 1,         # Send traces individually (not batched)
            "batch_timeout": 0.5,    # Send traces immediately
            "compression": False,    # No compression for easier debugging
            "verify_ssl": True       # Verify SSL certificates
        },
        
        # ✅ Comprehensive tracing configuration  
        tracing_config={
            "sample_rate": 1.0,        # Trace 100% of operations
            "capture_errors": True,    # Capture error details
            "auto_flush": True         # Automatically flush traces
        }
    )
    
    print("✅ Noveum Trace SDK initialized successfully!")
    
    # 📊 Display configuration details
    config = noveum_trace.get_config()
    print(f"📊 Project: {config.project}")
    print(f"🔧 Environment: {config.environment}")
    print(f"🌐 Transport Endpoint: {config.transport.endpoint}")
    print(f"📦 Batch Size: {config.transport.batch_size}")
    print(f"⏱️  Batch Timeout: {config.transport.batch_timeout}s")
    print(f"🔍 Debug Mode: {config.debug}")
    
    print("\\n🎯 SDK initialization completed!")
    print("📋 Check the debug log output above for HTTP request details")
    print("🌐 Your traces should now be visible at: https://noveum-trace.free.beeceptor.com")
    
except Exception as e:
    print(f"❌ Error initializing SDK: {e}")
    print("Continuing with demo - traces will be logged locally")
    import traceback
    traceback.print_exc()


\n🚀 Initializing Noveum Trace SDK with enhanced configuration...
🌐 Base Endpoint: https://noveum-trace.free.beeceptor.com
📡 Single Trace Endpoint: https://noveum-trace.free.beeceptor.com/v1/trace
📦 Batch Trace Endpoint: https://noveum-trace.free.beeceptor.com/v1/traces
🔧 Transport Mode: Individual traces (batch_size=1) for better debugging
✅ Noveum Trace SDK initialized successfully!
📊 Project: jupyter-test-project
🔧 Environment: development
🌐 Transport Endpoint: http://localhost:3000/api
📦 Batch Size: 100
⏱️  Batch Timeout: 5.0s
🔍 Debug Mode: True
\n🎯 SDK initialization completed!
📋 Check the debug log output above for HTTP request details
🌐 Your traces should now be visible at: https://noveum-trace.free.beeceptor.com


In [71]:
# 🧪 TEST TRACE SENDING TO VERIFY ENDPOINT CONNECTIVITY
print("🧪 Testing trace sending to verify endpoint connectivity...")

@trace
def test_endpoint_connectivity():
    """Test function to verify traces are being sent to the configured endpoint."""
    import time
    time.sleep(0.1)
    print("📡 Test trace function executed - this should generate HTTP requests")
    return {"test": "completed", "timestamp": time.time()}

# Execute test function
print("\\n🔄 Executing traced function...")
result = test_endpoint_connectivity()
print(f"✅ Function result: {result}")

# Force flush any pending traces
print("\\n📤 Forcing flush of traces...")
try:
    noveum_trace.flush()
    print("✅ Flush completed - traces should have been sent to endpoint")
except Exception as e:
    print(f"⚠️  Flush error: {e}")

# Summary
print("\\n📋 ENDPOINT DEBUGGING SUMMARY:")
print("🌐 Your Beeceptor endpoint: https://noveum-trace.free.beeceptor.com")
print("📡 SDK sends traces to:")
print("   - Single traces: https://noveum-trace.free.beeceptor.com/v1/trace")
print("   - Batch traces:  https://noveum-trace.free.beeceptor.com/v1/traces")
print("🔍 Check the debug logs above for HTTP request details")
print("📊 Check your Beeceptor dashboard to see if traces are arriving")

print("\\n✅ Enhanced debugging setup completed!")


DEBUG:noveum_trace.core.client:Started trace: 7d6cec38-2f53-46e0-99ae-5633c2bb2871
DEBUG:noveum_trace.core.client:Started span: ff75d7d0-16ca-4cbe-bbdd-f9c591309093 in trace: 7d6cec38-2f53-46e0-99ae-5633c2bb2871
DEBUG:noveum_trace.core.client:Finished span: ff75d7d0-16ca-4cbe-bbdd-f9c591309093
DEBUG:noveum_trace.transport.http_transport:Trace 7d6cec38-2f53-46e0-99ae-5633c2bb2871 queued for export
DEBUG:noveum_trace.core.client:Finished trace: 7d6cec38-2f53-46e0-99ae-5633c2bb2871
DEBUG:urllib3.connectionpool:Resetting dropped connection: localhost


🧪 Testing trace sending to verify endpoint connectivity...
\n🔄 Executing traced function...
📡 Test trace function executed - this should generate HTTP requests
✅ Function result: {'test': 'completed', 'timestamp': 1752952837.3284638}
\n📤 Forcing flush of traces...


DEBUG:urllib3.connectionpool:http://localhost:3000 "POST /api/v1/traces HTTP/1.1" 200 None
DEBUG:noveum_trace.transport.http_transport:Successfully sent batch of 1 traces
DEBUG:noveum_trace.transport.batch_processor:Sent batch of 1 traces
INFO:noveum_trace.transport.http_transport:HTTP transport flush completed
INFO:noveum_trace.core.client:Flushed all pending traces


✅ Flush completed - traces should have been sent to endpoint
\n📋 ENDPOINT DEBUGGING SUMMARY:
🌐 Your Beeceptor endpoint: https://noveum-trace.free.beeceptor.com
📡 SDK sends traces to:
   - Single traces: https://noveum-trace.free.beeceptor.com/v1/trace
   - Batch traces:  https://noveum-trace.free.beeceptor.com/v1/traces
🔍 Check the debug logs above for HTTP request details
📊 Check your Beeceptor dashboard to see if traces are arriving
\n✅ Enhanced debugging setup completed!


In [ ]:
# Import advanced features
from noveum_trace import (
    auto_instrument, get_instrumented_libraries, is_instrumented,
    create_traced_openai_client, create_traced_agent, TracedOpenAIClient,
    start_trace, start_span, get_current_trace, get_current_span
)

# Auto-Instrumentation Examples

def test_auto_instrumentation():
    """Test automatic instrumentation of libraries."""
    print("🔧 Testing Auto-Instrumentation...")
    
    # Check available instrumentations
    available = noveum_trace.get_available_instrumentations()
    print(f"📦 Available instrumentations: {available}")
    
    # Enable auto-instrumentation for OpenAI (if not already enabled)
    if not is_instrumented("openai"):
        print("🔌 Enabling OpenAI auto-instrumentation...")
        try:
            auto_instrument("openai")
            print("✅ OpenAI auto-instrumentation enabled")
        except Exception as e:
            print(f"⚠️  Auto-instrumentation: {e}")
    else:
        print("✅ OpenAI already instrumented")
    
    # Check instrumented libraries
    instrumented = get_instrumented_libraries()
    print(f"🔍 Currently instrumented: {instrumented}")
    
    return instrumented

# Proxy Objects for Enhanced Control

def test_traced_openai_client():
    """Test traced OpenAI client proxy."""
    print("🔄 Testing Traced OpenAI Client...")
    
    # Create traced OpenAI client (even without real API key)
    try:
        traced_client = create_traced_openai_client(
            api_key="mock-key-for-demo",
            trace_completions=True,
            trace_embeddings=True,
            capture_content=True
        )
        print("✅ Traced OpenAI client created")
        
        # Mock a call (won't actually work without real API key)
        print("🤖 Simulating traced OpenAI call...")
        # In real usage: response = traced_client.chat.completions.create(...)
        print("ℹ️  Would automatically trace all OpenAI API calls")
        
    except Exception as e:
        print(f"ℹ️  Traced client demo: {e}")

def test_traced_agent_proxy():
    """Test traced agent proxy for enhanced agent monitoring."""
    print("🤖 Testing Traced Agent Proxy...")
    
    # Mock agent class
    class MockAgent:
        def __init__(self, name: str):
            self.name = name
        
        def think(self, problem: str) -> str:
            time.sleep(0.2)
            return f"Thinking about: {problem}"
        
        def act(self, action: str) -> str:
            time.sleep(0.3)
            return f"Performing action: {action}"
        
        def plan(self, goal: str) -> List[str]:
            time.sleep(0.4)
            return [f"Step 1 for {goal}", f"Step 2 for {goal}", f"Step 3 for {goal}"]
    
    # Create traced agent proxy
    original_agent = MockAgent("demo_agent")
    traced_agent = create_traced_agent(
        agent=original_agent,
        agent_id="traced_demo_agent",
        auto_trace_methods=["think", "act", "plan"],
        capture_inputs=True,
        capture_outputs=True
    )
    
    print("✅ Traced agent proxy created")
    
    # Test traced methods
    print("🧠 Testing traced agent methods...")
    
    thought = traced_agent.think("How to improve AI observability")
    print(f"💭 Think result: {thought}")
    
    action = traced_agent.act("Implement monitoring dashboard")
    print(f"⚡ Action result: {action}")
    
    plan = traced_agent.plan("Enhance system reliability")
    print(f"📋 Plan result: {len(plan)} steps")

# Manual Span Creation and Management

def test_manual_tracing():
    """Test manual trace and span creation."""
    print("🔍 Testing Manual Tracing...")
    
    # Start a manual trace
    trace = start_trace("manual_demo_trace")
    print(f"✅ Started trace: {trace.trace_id}")
    
    # Create nested spans manually
    with trace.span("parent_operation") as parent_span:
        parent_span.set_attributes({
            "operation.type": "parent",
            "operation.importance": "high"
        })
        print("📊 Parent span created")
        
        # Child span 1
        with parent_span.create_child_span("child_operation_1") as child1:
            child1.set_attributes({
                "operation.type": "child",
                "child.number": 1
            })
            time.sleep(0.2)
            print("🔹 Child span 1 completed")
        
        # Child span 2  
        with parent_span.create_child_span("child_operation_2") as child2:
            child2.set_attributes({
                "operation.type": "child",
                "child.number": 2,
                "child.data_processed": 500
            })
            time.sleep(0.3)
            print("🔹 Child span 2 completed")
        
        print("📊 Parent operation completed")
    
    # Finish trace
    trace.finish()
    print(f"✅ Manual trace completed: {trace.trace_id}")

# Advanced Configuration and Performance Features

def test_advanced_configuration():
    """Test advanced SDK configuration options."""
    print("⚙️  Testing Advanced Configuration...")
    
    # Get current configuration
    config = noveum_trace.get_config()
    print(f"📋 Current project: {config.project}")
    print(f"🌐 Current endpoint: {config.transport.endpoint}")
    print(f"📦 Batch size: {config.transport.batch_size}")
    print(f"⏱️  Batch timeout: {config.transport.batch_timeout}")
    
    # Test configuration updates (temporary for demo)
    original_debug = config.debug
    
    # Temporarily enable debug mode
    noveum_trace.configure(debug=True)
    print("🐛 Debug mode enabled temporarily")
    
    # Create a trace to demonstrate debug output
    with noveum_trace.trace_operation("debug_demo_operation") as span:
        span.set_attributes({"demo": "configuration", "debug_enabled": True})
        time.sleep(0.1)
        print("✅ Debug operation completed")
    
    # Restore original debug setting
    noveum_trace.configure(debug=original_debug)
    print(f"🔧 Debug mode restored to: {original_debug}")

# Batch Processing and Performance Monitoring

def test_batch_processing():
    """Test batch operations for performance."""
    print("📦 Testing Batch Processing...")
    
    # Create multiple operations quickly to test batching
    operations = []
    
    for i in range(5):
        with noveum_trace.trace_operation(f"batch_operation_{i}", operation_type="batch_demo") as span:
            span.set_attributes({
                "batch.operation_number": i,
                "batch.total_operations": 5,
                "operation.size": "small"
            })
            time.sleep(0.05)  # Quick operations
            operations.append(f"operation_{i}")
            print(f"🔸 Batch operation {i+1}/5 completed")
    
    print(f"✅ Batch processing completed: {len(operations)} operations")
    
    # Force flush to send batched traces
    try:
        noveum_trace.flush()
        print("📤 Forced flush of batched traces")
    except Exception as e:
        print(f"ℹ️  Flush status: {e}")

# Error Handling and Edge Cases

def test_error_handling():
    """Test error handling and edge cases."""
    print("⚠️  Testing Error Handling...")
    
    # Test error capture in traced function
    @noveum_trace.trace(capture_errors=True, capture_stack_trace=True)
    def operation_with_error(should_fail: bool = False):
        if should_fail:
            raise ValueError("This is a demo error for testing")
        return "Success!"
    
    # Test successful operation
    try:
        result = operation_with_error(should_fail=False)
        print(f"✅ Successful operation: {result}")
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
    
    # Test error capture
    try:
        result = operation_with_error(should_fail=True)
        print(f"Unexpected success: {result}")
    except ValueError as e:
        print(f"✅ Error captured successfully: {e}")

# Run all advanced feature tests
print("🚀 Testing Auto-Instrumentation and Advanced Features...")

print("\\n1️⃣ Auto-Instrumentation:")
instrumented_libs = test_auto_instrumentation()

print("\\n2️⃣ Proxy Objects:")
test_traced_openai_client()
test_traced_agent_proxy()

print("\\n3️⃣ Manual Tracing:")
test_manual_tracing()

print("\\n4️⃣ Advanced Configuration:")
test_advanced_configuration()

print("\\n5️⃣ Batch Processing:")
test_batch_processing()

print("\\n6️⃣ Error Handling:")
test_error_handling()

print("\\n✅ Advanced features testing completed!")


In [ ]:
@trace_agent(agent_id="research_agent")
def research_agent(query: str) -> Dict[str, Any]:
    """Research agent that gathers information."""
    print(f"🔍 Research Agent: Processing query '{query}'")

    # Simulate research process
    time.sleep(0.4)

    # Mock research findings
    findings = {
        "query": query,
        "sources": ["source1.pdf", "source2.html", "source3.json"],
        "key_points": [
            "Point 1: Observability improves system reliability",
            "Point 2: Tracing helps identify bottlenecks",
            "Point 3: Monitoring enables proactive maintenance"
        ],
        "confidence": 0.87,
        "research_time": "0.4s"
    }

    print(f"✅ Research completed with {len(findings['sources'])} sources")
    return findings

@trace_agent(agent_id="orchestrator")
def orchestrate_workflow(task: str) -> Dict[str, Any]:
    """Orchestrator agent that coordinates multiple agents."""
    print(f"🎭 Orchestrator: Starting workflow for task '{task}'")

    # Step 1: Research
    print("\n🔍 Step 1: Research Phase")
    research_data = research_agent(task)

    # Step 2: Analysis (simplified)
    print("\n📊 Step 2: Analysis Phase")
    analysis_data = {
        "insights": ["Observability is crucial", "Tracing provides insights"],
        "quality_score": 0.89
    }

    # Final orchestration result
    workflow_result = {
        "task": task,
        "workflow_id": "wf-001",
        "phases_completed": 2,
        "research_summary": research_data["key_points"],
        "analysis_summary": analysis_data["insights"],
        "overall_confidence": research_data["confidence"],
        "total_time": "1.0s"
    }

    print("\n✅ Orchestrator: Workflow completed successfully")
    return workflow_result

# Test the multi-agent workflow
task = "Analyze the importance of observability in AI systems"
workflow_result = orchestrate_workflow(task)
print("\n🎭 Final Workflow Result:")
print(f"Task: {workflow_result['task']}")
print(f"Confidence: {workflow_result['overall_confidence']:.2f}")
print(f"Phases: {workflow_result['phases_completed']}")


In [ ]:
@trace_tool(tool_name="calculator")
def calculate(operation: str, a: float, b: float) -> Dict[str, Any]:
    """A calculator tool with tracing."""
    print(f"🔢 Calculator: Performing {operation} on {a} and {b}")

    operations = {
        "add": lambda x, y: x + y,
        "subtract": lambda x, y: x - y,
        "multiply": lambda x, y: x * y,
        "divide": lambda x, y: x / y if y != 0 else None
    }

    if operation not in operations:
        return {"error": f"Unknown operation: {operation}"}

    try:
        result = operations[operation](a, b)
        if result is None:
            return {"error": "Division by zero"}

        return {
            "operation": operation,
            "operands": [a, b],
            "result": result,
            "success": True
        }
    except Exception as e:
        return {"error": str(e), "success": False}

@trace_tool(tool_name="text_analyzer")
def analyze_text(text: str) -> Dict[str, Any]:
    """Text analysis tool with tracing."""
    print(f"📝 Text Analyzer: Analyzing text of length {len(text)}")

    # Simulate analysis
    time.sleep(0.2)

    analysis = {
        "text_length": len(text),
        "word_count": len(text.split()),
        "sentence_count": text.count('.') + text.count('!') + text.count('?'),
        "avg_word_length": sum(len(word) for word in text.split()) / len(text.split()) if text.split() else 0
    }

    print(f"✅ Analysis complete: {analysis['word_count']} words, {analysis['sentence_count']} sentences")
    return analysis

# Test tool tracing
calc_result = calculate("multiply", 15, 4)
print(f"\n🔢 Calculator Result: {calc_result}")

text_to_analyze = "This is a sample text for testing the noveum-trace SDK. It contains multiple sentences!"
text_analysis = analyze_text(text_to_analyze)
print(f"\n📝 Text Analysis Result: {text_analysis}")


In [ ]:
import sys
import pkg_resources

# Test summary
def print_test_summary():
    """Print a summary of all tests performed."""
    print("📋 NOVEUM TRACE SDK TEST SUMMARY")
    print("=" * 50)

    # Check SDK version
    try:
        version = pkg_resources.get_distribution("noveum-trace").version
        print(f"✅ SDK Version: {version}")
    except:
        print("⚠️  Could not determine SDK version")

    # Environment check
    print(f"✅ Python Version: {sys.version.split()[0]}")
    print(f"✅ Environment Variables: {'✓' if os.getenv('NOVEUM_API_KEY') else '✗'}")

    # Features tested
    features_tested = [
        "Basic function tracing (@trace)",
        "LLM call tracing (@trace_llm)",
        "Agent workflow tracing (@trace_agent)",
        "Tool tracing (@trace_tool)",
        "Multi-agent orchestration",
        "Error handling",
        "Framework integration simulation"
    ]

    print("\n🧪 Features Tested:")
    for feature in features_tested:
        print(f"  ✅ {feature}")

    print("\n🎯 Key Results:")
    print("  🔧 All decorators: Functional")
    print("  🤖 Multi-agent support: Functional")
    print("  🔌 Framework integration: Simulated successfully")

    print("\n✅ All tests completed successfully!")
    print("\n📖 Next Steps:")
    print("  1. Set up your actual NOVEUM_API_KEY for production use")
    print("  2. Integrate with your LLM applications")
    print("  3. Set up dashboards and monitoring")
    print("  4. Configure alerting based on trace data")

# Clean up function
def cleanup_resources():
    """Clean up any resources created during testing."""
    print("🧹 Cleaning up test resources...")

    try:
        # Attempt to flush any pending traces
        noveum_trace.flush()
        print("✅ Traces flushed successfully")
    except Exception as e:
        print(f"ℹ️  Trace flush: {e}")

    print("✅ Cleanup completed")

# Run summary and cleanup
print_test_summary()
cleanup_resources()
